# Step 1: Load Transcript Data

**Input Files:**
- `course_material/transcripts/*.p` (pickle files for each company)

**Output:**
- Variable: `transcripts` (list of transcript dictionaries)

This script loads transcript data for use in subsequent analysis steps.

In [1]:
import os
import json
import pickle
from typing import List, Dict, Any
import pandas as pd

# Configuration
OUT_FIG    = "exports/figures"
OUT_TAB    = "exports/tables"
REPORT_DIR = "exports/JsonFile"

for d in [OUT_FIG, OUT_TAB, REPORT_DIR]:
    os.makedirs(d, exist_ok=True)
# Define Group 7 tickers
TickersOfCompany = [
    'UNH', 'MET', 'HD', 'BKNG', 'MCO', 'ABT', 'INTU', 'COST', 
    'TSM', 'NKE', 'LLY', 'AZN', 'MS', 'BAC', 'F', 'TMO', 'V', 
    'QCOM', 'AXP', 'MCD', 'IBM', 'JNJ', 'DUK', 'ISRG', 'XOM', 
    'ASML', 'FB', 'INTC', 'AMT', 'FDX', 'ORCL', 'BA', 'MA', 
    'CRM', 'T', 'TSLA', 'MDT', 'UNP', 'PEP', 'BLK', 'AMGN', 
    'SAP', 'BMY', 'NFLX', 'TMUS', 'HON', 'LMT', 'SNY', 'ADBE', 
    'JPM', 'TD', 'EBAY', 'GS', 'PM', 'CMCSA', 'GM', 'PG', 'GOOG', 'UPS', 'MSFT'
]
# TickersOfCompany = [
#      'MSFT', 'INTC', 'NFLX', 'MA', 'V','JPM','BMY','JNJ', 'AMGN', 'XOM','UNH'
# ]

def load_transcripts(transcript_folder: str, tickers: List[str]) -> List[Dict[str, Any]]:
    """
    Load transcripts for specified tickers
    
    Args:
        transcript_folder (str): Path to folder containing .p files
        tickers (List[str]): List of ticker symbols to load
        
    Returns:
        List[Dict[str, Any]]: List of transcript dictionaries
    """
    transcriptsDATACompany = []
    success_count = 0
    error_count = 0
    
    print("Loading transcript data...")
    print("=" * 60)
    
    for ticker in tickers:
        try:
            file_path = os.path.join(transcript_folder, f"{ticker}.p")
            
            # Load pickle file
            with open(file_path, "rb") as f:
                data = pickle.load(f)
            
            transcriptsDATACompany.extend(data)
            success_count += 1
            print(f"✓ Loaded {len(data)} transcripts for {ticker}")
            
        except FileNotFoundError:
            error_count += 1
            print(f"✗ No transcript file found for {ticker}")
            
        except Exception as e:
            error_count += 1
            print(f"✗ Error loading {ticker}: {str(e)}")
    
    print("=" * 60)
    print(f"Total transcripts loaded: {len(transcriptsDATACompany)}")
    print(f"Companies with data: {success_count}")
    print(f"Companies missing data: {error_count}")
    
    return transcriptsDATACompany

def step1_load_transcripts():
    """Main function to load transcript data"""
    print("=" * 80)
    print("STEP 1: LOADING TRANSCRIPT DATA")
    print("=" * 80)
    
    transcripts = load_transcripts("course_material/transcripts", TickersOfCompany)
    
    if not transcripts:
        print("No transcripts loaded. Please check file paths.")
        return None
    
    print(f"Successfully loaded {len(transcripts)} transcripts")
    print("Transcripts are now available in memory for use in subsequent cells")
    
    return transcripts

# Load transcripts and store in variable for use in subsequent cells
transcripts = step1_load_transcripts()

STEP 1: LOADING TRANSCRIPT DATA
Loading transcript data...
✓ Loaded 57 transcripts for UNH
✓ Loaded 65 transcripts for MET
✓ Loaded 89 transcripts for HD
✓ Loaded 61 transcripts for BKNG
✓ Loaded 61 transcripts for MCO
✓ Loaded 68 transcripts for ABT
✓ Loaded 64 transcripts for INTU
✓ Loaded 65 transcripts for COST
✓ Loaded 61 transcripts for TSM
✓ Loaded 59 transcripts for NKE
✓ Loaded 77 transcripts for LLY
✓ Loaded 45 transcripts for AZN
✓ Loaded 65 transcripts for MS
✓ Loaded 77 transcripts for BAC
✓ Loaded 142 transcripts for F
✓ Loaded 67 transcripts for TMO
✓ Loaded 97 transcripts for V
✓ Loaded 82 transcripts for QCOM
✓ Loaded 72 transcripts for AXP
✓ Loaded 77 transcripts for MCD
✓ Loaded 83 transcripts for IBM
✓ Loaded 91 transcripts for JNJ
✓ Loaded 61 transcripts for DUK
✓ Loaded 55 transcripts for ISRG
✓ Loaded 76 transcripts for XOM
✓ Loaded 70 transcripts for ASML
✓ Loaded 46 transcripts for FB
✓ Loaded 90 transcripts for INTC
✓ Loaded 63 transcripts for AMT
✓ Loaded 52 

# Step 2: Extract Event Dates

**Input:**
- Variable: `transcripts` (from Step 1)

**Output Files:**
- `exports/JsonFile/event_dates.json`
- `exports/tables/WRDS_Event_Dates.csv`
- `exports/tables/Event_Dates_Simple.txt`
This script extracts event dates from loaded transcripts for WRDS upload.
Uses transcripts variable directly from previous cell.

In [2]:
import os
import pandas as pd
import json
from datetime import datetime
from typing import List, Dict, Any

# Configuration
OUT_TAB = "exports/tables"
JSON_DIR = "exports/JsonFile"

def extract_events(transcripts: List[Dict[str, Any]]) -> pd.DataFrame:
    """
    Extract event information from transcripts
    
    Args:
        transcripts (List[Dict]): List of transcript dictionaries
        
    Returns:
        pd.DataFrame: DataFrame with event information
    """
    print("Extracting event dates and information...")
    print("=" * 60)
    
    event_data = []
    
    for transcript in transcripts:
        try:
            event_info = {
                'ticker': transcript.get('symbol', ''),
                'quarter': transcript.get('quarter', ''),
                'fiscal_year': transcript.get('year', ''),
                'event_date': transcript.get('time', ''),
                'event_type': 'Earnings Call',
                'transcript_id': transcript.get('id', ''),
                'title': transcript.get('title', ''),
                'audio_url': transcript.get('audio', ''),
                'participants_count': len(transcript.get('participant', [])) if transcript.get('participant') else 0,
                'transcript_sections': len(transcript.get('transcript', [])) if transcript.get('transcript') else 0
            }
            event_data.append(event_info)
            
        except Exception as e:
            print(f"  Error processing transcript: {str(e)}")
            continue
    
    # Convert to DataFrame
    events_df = pd.DataFrame(event_data)
    
    if len(events_df) > 0:
        # Convert event_date to proper datetime format
        events_df['event_date'] = pd.to_datetime(events_df['event_date'])
        
        # Sort by ticker and date
        events_df = events_df.sort_values(['ticker', 'event_date'])
        
        # Add formatted date columns
        events_df['event_date_formatted'] = events_df['event_date'].dt.strftime('%Y-%m-%d')
        events_df['year'] = events_df['event_date'].dt.year
        events_df['month'] = events_df['event_date'].dt.month
        events_df['day'] = events_df['event_date'].dt.day
        events_df['quarter_year'] = events_df['fiscal_year'].astype(str) + 'Q' + events_df['quarter'].astype(str)
        
        # Remove duplicate events (same ticker and date)
        pre_dedup_count = len(events_df)
        events_df = events_df.drop_duplicates(subset=['ticker', 'event_date_formatted'], keep='first')
        post_dedup_count = len(events_df)
        duplicates_removed = pre_dedup_count - post_dedup_count
        
        print(f"✓ Extracted {pre_dedup_count} events")
        if duplicates_removed > 0:
            print(f"✓ Removed {duplicates_removed} duplicate events")
            print(f"✓ Final unique events: {post_dedup_count}")
        print(f"✓ Date range: {events_df['event_date'].min()} to {events_df['event_date'].max()}")
        print(f"✓ Unique companies: {events_df['ticker'].nunique()}")
        
    else:
        print("  No events extracted")
        
    return events_df

def save_for_wrds(events_df: pd.DataFrame):
    """
    Save event data in formats suitable for WRDS upload
    
    Args:
        events_df (pd.DataFrame): DataFrame with event information
    """
    if events_df.empty:
        print("  No events to save")
        return
    
    print("\nSaving event data for WRDS...")
    print("=" * 60)
    
    # 1. WRDS-compatible CSV format
    wrds_format = events_df[['ticker', 'event_date_formatted', 'event_type']].copy()
    wrds_format.columns = ['Ticker', 'Event_Date', 'Event_Type']
    
    wrds_csv_path = os.path.join(OUT_TAB, "WRDS_Event_Dates.csv")
    wrds_format.to_csv(wrds_csv_path, index=False)
    print(f"✓ WRDS CSV saved to: {wrds_csv_path}")
    
    # 2. Comprehensive Excel file with multiple sheets
    excel_path = os.path.join(OUT_TAB, "Event_Analysis_Data.xlsx")
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Full event data
        events_df.to_excel(writer, sheet_name='All_Events', index=False)
        
        # WRDS format
        wrds_format.to_excel(writer, sheet_name='WRDS_Format', index=False)
        
        # Summary by company
        summary_by_company = events_df.groupby('ticker').agg({
            'event_date': ['count', 'min', 'max'],
            'quarter': 'nunique',
            'participants_count': 'mean',
            'transcript_sections': 'mean'
        }).round(2)
        summary_by_company.columns = ['Total_Events', 'First_Event', 'Last_Event', 
                                    'Unique_Quarters', 'Avg_Participants', 'Avg_Sections']
        summary_by_company.to_excel(writer, sheet_name='Company_Summary')
        
        # Summary by year
        summary_by_year = events_df.groupby('year').agg({
            'ticker': 'nunique',
            'event_date': 'count',
            'participants_count': 'mean',
            'transcript_sections': 'mean'
        }).round(2)
        summary_by_year.columns = ['Unique_Companies', 'Total_Events', 
                                 'Avg_Participants', 'Avg_Sections']
        summary_by_year.to_excel(writer, sheet_name='Year_Summary')
        
        # Summary by quarter
        summary_by_quarter = events_df.groupby(['year', 'quarter']).agg({
            'ticker': 'nunique',
            'event_date': 'count'
        }).round(2)
        summary_by_quarter.columns = ['Unique_Companies', 'Total_Events']
        summary_by_quarter.to_excel(writer, sheet_name='Quarter_Summary')
    
    print(f"✓ Excel file saved to: {excel_path}")
    
    # 3. JSON format for programmatic access
    json_path = os.path.join(JSON_DIR, "event_dates.json")
    events_json = events_df.to_dict('records')
    with open(json_path, 'w') as f:
        json.dump(events_json, f, indent=2, default=str)
    print(f"✓ JSON format saved to: {json_path}")
    
    # 4. Simple text format for quick reference
    txt_path = os.path.join(OUT_TAB, "Event_Dates_Simple.txt")
    with open(txt_path, 'w') as f:
        f.write("TICKER\tEVENT_DATE\tQUARTER\tFISCAL_YEAR\n")
        for _, row in events_df.iterrows():
            f.write(f"{row['ticker']}\t{row['event_date_formatted']}\t"
                   f"Q{row['quarter']}\t{row['fiscal_year']}\n")
    print(f"✓ Text format saved to: {txt_path}")
    
    # Print summary statistics and instructions
    print_summary_statistics(events_df)
    print_wrds_instructions(wrds_csv_path)

def print_summary_statistics(events_df: pd.DataFrame):
    """Print summary statistics about the events"""
    print("\n" + "=" * 60)
    print("EVENT SUMMARY STATISTICS")
    print("=" * 60)
    
    print(f"Total events: {len(events_df)}")
    print(f"Unique companies: {events_df['ticker'].nunique()}")
    print(f"Date range: {events_df['event_date'].min().strftime('%Y-%m-%d')} to {events_df['event_date'].max().strftime('%Y-%m-%d')}")
    print(f"Events per company (average): {len(events_df) / events_df['ticker'].nunique():.1f}")
    
    print("\nEvents by year:")
    year_counts = events_df['year'].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"  {year}: {count} events")
    
    print(f"\nTop 10 companies by number of events:")
    company_counts = events_df['ticker'].value_counts().head(10)
    for ticker, count in company_counts.items():
        print(f"  {ticker}: {count} events")
    
    print(f"\nSample event dates (first 10):")
    sample_events = events_df[['ticker', 'event_date_formatted', 'quarter', 'fiscal_year']].head(10)
    print(sample_events.to_string(index=False))

def print_wrds_instructions(wrds_csv_path: str):
    """Print instructions for using WRDS Event Study"""
    print("\n" + "=" * 60)
    print("INSTRUCTIONS FOR WRDS EVENT STUDY")
    print("=" * 60)
    print(f"1. Upload file: {wrds_csv_path}")
    print("2. WRDS Event Study Settings:")
    print("   - Estimation Window: -250 to -46 days (before event)")
    print("   - Event Window: -1 to +1 days (around event)")
    print("   - Market Index: CRSP Value-Weighted Index")
    print("   - Return Type: Daily returns")
    print("3. Download CAR results with t-statistics")
    print("4. Recommended event windows to test:")
    print("   - CAR(-1,+1): Three-day window around announcement")
    print("   - CAR(0,+1): Two-day window from announcement")
    print("   - CAR(-2,+2): Five-day window for robustness")
    print("=" * 60)

def step2_extract_event_dates(transcripts_data):
    """Main function to extract event dates"""
    print("=" * 80)
    print("STEP 2: EXTRACTING EVENT DATES FOR WRDS")
    print("=" * 80)
    
    # Check if transcripts data is available
    if not transcripts_data:
        print("[ERROR] No transcript data available.")
        print("Please run the previous cell (Step 1) first to load transcripts.")
        return None
    
    print(f"[OK] Using {len(transcripts_data)} transcripts from memory")
    
    # Extract and save events
    event_data = extract_events(transcripts_data)
    save_for_wrds(event_data)
    
    print("[OK] Event dates extracted and saved for WRDS")
    print("Files created:")
    print("  - WRDS_Event_Dates.csv (Upload this to WRDS)")
    print("  - Event_Analysis_Data.xlsx (Detailed analysis)")
    
    return event_data

# Extract event dates using transcripts from previous cell
if 'transcripts' in locals() and transcripts:
    event_data = step2_extract_event_dates(transcripts)
else:
    print("[WARNING] Transcripts not found. Please run Step 1 first.")
    event_data = None

STEP 2: EXTRACTING EVENT DATES FOR WRDS
[OK] Using 4373 transcripts from memory
Extracting event dates and information...
✓ Extracted 4373 events
✓ Removed 25 duplicate events
✓ Final unique events: 4348
✓ Date range: 2005-10-31 00:40:28 to 2021-03-12 01:38:07
✓ Unique companies: 60

Saving event data for WRDS...
✓ WRDS CSV saved to: exports/tables\WRDS_Event_Dates.csv
✓ Excel file saved to: exports/tables\Event_Analysis_Data.xlsx
✓ JSON format saved to: exports/JsonFile\event_dates.json
✓ Text format saved to: exports/tables\Event_Dates_Simple.txt

EVENT SUMMARY STATISTICS
Total events: 4348
Unique companies: 60
Date range: 2005-10-31 to 2021-03-12
Events per company (average): 72.5

Events by year:
  2005: 9 events
  2006: 104 events
  2007: 167 events
  2008: 232 events
  2009: 212 events
  2010: 181 events
  2011: 247 events
  2012: 304 events
  2013: 445 events
  2014: 353 events
  2015: 347 events
  2016: 334 events
  2017: 359 events
  2018: 367 events
  2019: 347 events
  2020:

# Step 3: Sentiment Analysis

**Input Files:**
- `Loughran-McDonald_MasterDictionary_1993-2024.csv`
- Variable: `transcripts` (from Step 1)

**Output Files:**
- `exports/JsonFile/sentiment_results.json`
- `exports/tables/Sentiment_Analysis_Results.csv`
This script performs sentiment analysis on loaded transcripts using spaCy and 
Loughran-McDonald dictionary following the proper pattern from reference notebooks.
Uses transcripts variable directly from cell 1.

In [3]:
import os
import pandas as pd
import numpy as np
import spacy
from collections import Counter
import json
import re

# Load spaCy model
!python -m spacy download en_core_web_md # Ensure model is downloaded
try:
    nlp = spacy.load('en_core_web_md')
    print("✓ spaCy model loaded successfully")
except OSError:
    print("[ERROR] spaCy model 'en_core_web_md' not found.")
    print("Please install it using: python -m spacy download en_core_web_md")
    raise

def process_spacy_doc(text):
    """
    Process text using spaCy for financial sentiment analysis.
    This is the EXACT function from the reference notebook.
    """
    words = [
        word.lemma_.lower()                 # Lemmatize and lowercase
        for word in nlp(text)              # Tokenize with spaCy
        if not (
            word.is_space                   # Remove spaces
            or word.is_stop                 # Remove stop words
            or word.is_punct                # Remove punctuation
        )
    ]
    return words

def extract_transcript_text(transcript):
    """
    Extract all speech text from a transcript dictionary.
    Following the exact pattern from reference notebook.
    """
    text_transcript = ''
    
    # Loop through all speech segments
    for speech in transcript['transcript']:
        # Extract the speech text (first element of the speech list)
        speech_text = speech['speech'][0]
        
        # Add to our full text with space separator
        text_transcript = text_transcript + ' ' + speech_text
    
    return text_transcript

def load_loughran_mcdonald_dictionary():
    """
    Load Loughran-McDonald dictionary and process with spaCy
    Following the exact pattern from reference notebook.
    """
    print("Loading Loughran-McDonald sentiment dictionary...")
    
    # Try both possible file names
    excel_files = [
        "LoughranMcDonald_SentimentWordLists_2018.xlsx"
    ]
    
    dictionary_loaded = False
    
    # Try Excel format first (preferred method from reference)
    for excel_file in excel_files:
        if os.path.exists(excel_file):
            if excel_file.endswith('.xlsx'):
                try:
                    print(f"✓ Loading from Excel file: {excel_file}")
                    
                    LM_negative = pd.read_excel(excel_file, sheet_name='Negative', header=None)
                    LM_positive = pd.read_excel(excel_file, sheet_name='Positive', header=None)
                    LM_uncertainty = pd.read_excel(excel_file, sheet_name='Uncertainty', header=None)
                    
                    # Convert DataFrames to lists (simplified approach from reference)
                    LM_positive = list(LM_positive[0])     # Get first column as list
                    LM_negative = list(LM_negative[0])     # Get first column as list  
                    LM_uncertainty = list(LM_uncertainty[0])  # Get first column as list
                    
                    print("✓ Converted to Python lists")
                    
                    # Process with spaCy for consistency (from reference)
                    LM_positive = process_spacy_doc(' '.join(LM_positive))
                    LM_negative = process_spacy_doc(' '.join(LM_negative))
                    LM_uncertainty = process_spacy_doc(' '.join(LM_uncertainty))
                    
                    print("✓ Processed with spaCy (lemmatized, lowercase)")
                    
                    # Convert to sets for fast lookup
                    LM_positive = set(LM_positive)
                    LM_negative = set(LM_negative)
                    LM_uncertainty = set(LM_uncertainty)
                    
                    print(f"✓ Converted to sets for fast word matching")
                    print(f"✓ Final counts - Positive: {len(LM_positive)}, Negative: {len(LM_negative)}, Uncertainty: {len(LM_uncertainty)}")
                    
                    dictionary_loaded = True
                    break
                    
                except Exception as e:
                    print(f"  Could not load from {excel_file}: {e}")
                    continue
                    
            elif excel_file.endswith('.csv'):
                try:
                    print(f"✓ Loading from CSV file: {excel_file}")
                    
                    # Load CSV format
                    df = pd.read_csv(excel_file)
                    
                    # Extract sentiment categories
                    pos_words = df[df['Positive'] > 0]['Word'].str.lower().tolist()
                    neg_words = df[df['Negative'] > 0]['Word'].str.lower().tolist()
                    unc_words = df[df['Uncertainty'] > 0]['Word'].str.lower().tolist()
                    
                    # Process with spaCy
                    LM_positive = set(process_spacy_doc(' '.join(pos_words)))
                    LM_negative = set(process_spacy_doc(' '.join(neg_words)))
                    LM_uncertainty = set(process_spacy_doc(' '.join(unc_words)))
                    
                    print(f"✓ Final counts - Positive: {len(LM_positive)}, Negative: {len(LM_negative)}, Uncertainty: {len(LM_uncertainty)}")
                    
                    dictionary_loaded = True
                    break
                    
                except Exception as e:
                    print(f"  Could not load from {excel_file}: {e}")
                    continue
    
    if not dictionary_loaded:
        raise FileNotFoundError(
            "Could not find Loughran-McDonald dictionary. Please ensure one of these files exists:\n"
            "- LoughranMcDonald_SentimentWordLists_2018.xlsx\n"
        )
    
    return LM_positive, LM_negative, LM_uncertainty

def analyze_transcript_sentiment(transcript, LM_positive, LM_negative, LM_uncertainty):
    """
    Analyze sentiment for a single transcript.
    Following the exact pattern from reference notebook.
    """
    # Extract text using the exact pattern from reference
    text_transcript = extract_transcript_text(transcript)
    
    # Process with spaCy
    processed_words = process_spacy_doc(text_transcript)
    total_words = len(processed_words)
    
    if total_words == 0:
        return {
            'ticker': transcript.get('symbol', ''),
            'quarter': transcript.get('quarter', ''),
            'fiscal_year': transcript.get('year', ''),
            'event_date': transcript.get('time', ''),
            'transcript_id': transcript.get('id', ''),
            'total_words': 0,
            'positive_count': 0,
            'negative_count': 0,
            'uncertainty_count': 0,
            'positive_ratio': 0.0,
            'negative_ratio': 0.0,
            'uncertainty_ratio': 0.0,
            'net_sentiment': 0.0,
            'sentiment_polarity': 0.0
        }
    
    # Count sentiment words using sets for fast lookup
    pos_count = sum(1 for word in processed_words if word in LM_positive)
    neg_count = sum(1 for word in processed_words if word in LM_negative)
    unc_count = sum(1 for word in processed_words if word in LM_uncertainty)
    
    # Calculate ratios
    pos_ratio = pos_count / total_words
    neg_ratio = neg_count / total_words
    unc_ratio = unc_count / total_words
    
    # Calculate composite measures
    net_sentiment = pos_ratio - neg_ratio
    
    # Sentiment polarity: (Positive - Negative) / (Positive + Negative)
    pos_neg_sum = pos_count + neg_count
    sentiment_polarity = (pos_count - neg_count) / pos_neg_sum if pos_neg_sum > 0 else 0.0
    
    return {
        'ticker': transcript.get('symbol', ''),
        'quarter': transcript.get('quarter', ''),
        'fiscal_year': transcript.get('year', ''),
        'event_date': transcript.get('time', ''),
        'transcript_id': transcript.get('id', ''),
        'total_words': total_words,
        'positive_count': pos_count,
        'negative_count': neg_count,
        'uncertainty_count': unc_count,
        'positive_ratio': pos_ratio,
        'negative_ratio': neg_ratio,
        'uncertainty_ratio': unc_ratio,
        'net_sentiment': net_sentiment,
        'sentiment_polarity': sentiment_polarity
    }

def step3_sentiment_analysis_with_spacy(transcripts_data):
    """
    Perform sentiment analysis on transcripts one by one using spaCy.
    Following memory-efficient pattern from reference notebooks.
    """
    print("=" * 80)
    print("STEP 3: SENTIMENT ANALYSIS WITH SPACY")
    print("=" * 80)
    
    # Check if transcripts data is available
    if not transcripts_data:
        print("[ERROR] No transcript data available.")
        print("Please run Step 1 first to load transcripts.")
        return None
    
    print(f"[OK] Using {len(transcripts_data)} transcripts from memory")
    
    # Load Loughran-McDonald dictionary with spaCy processing
    try:
        LM_positive, LM_negative, LM_uncertainty = load_loughran_mcdonald_dictionary()
    except Exception as e:
        print(f"[ERROR] Could not load sentiment dictionary: {e}")
        return None
    
    # Process transcripts one by one (memory-efficient approach)
    print("\n=== PROCESSING TRANSCRIPTS ONE BY ONE ===")
    
    results = []
    total_transcripts = len(transcripts_data)
    
    for i, transcript in enumerate(transcripts_data):
        try:
            # Analyze sentiment for this transcript
            result = analyze_transcript_sentiment(transcript, LM_positive, LM_negative, LM_uncertainty)
            results.append(result)
            
            # Progress indicator (every 50 transcripts or first few)
            if (i + 1) % 50 == 0 or i < 5 or i == total_transcripts - 1:
                print(f"✓ Processed {i + 1}/{total_transcripts}: {result['ticker']} Q{result['quarter']}")
                print(f"  Pos: {result['positive_ratio']:.4f}, Neg: {result['negative_ratio']:.4f}, Net: {result['net_sentiment']:.4f}")
            
            # Clear variables to free memory (following reference pattern)
            transcript = None
            result = None
            
        except Exception as e:
            print(f"  [ERROR] Processing transcript {i}: {str(e)}")
            continue
    
    # Convert to DataFrame
    sentiment_df = pd.DataFrame(results)
    
    if not sentiment_df.empty:
        # Convert event_date to datetime
        sentiment_df['event_date'] = pd.to_datetime(sentiment_df['event_date'])
        
        # Sort by ticker and date
        sentiment_df = sentiment_df.sort_values(['ticker', 'event_date'])
        
        print(f"\n✓ Sentiment analysis completed: {len(sentiment_df)} transcripts processed")
        
        # Print summary statistics
        print(f"\n=== SENTIMENT SUMMARY ===")
        print(f"Average Positive Ratio: {sentiment_df['positive_ratio'].mean():.4f}")
        print(f"Average Negative Ratio: {sentiment_df['negative_ratio'].mean():.4f}")
        print(f"Average Uncertainty Ratio: {sentiment_df['uncertainty_ratio'].mean():.4f}")
        print(f"Average Net Sentiment: {sentiment_df['net_sentiment'].mean():.4f}")
        
    else:
        print("[ERROR] No sentiment results generated")
        return None
    
    # Save results
    save_sentiment_results(sentiment_df)
    
    return sentiment_df

def save_sentiment_results(sentiment_df):
    """Save sentiment analysis results to files"""
    
    # Configuration
    OUT_TAB = "exports/tables"
    JSON_DIR = "exports/JsonFile"
    
    print("\n=== SAVING RESULTS ===")
    
    # 1. CSV format for analysis
    csv_path = os.path.join(OUT_TAB, "Sentiment_Analysis_Results.csv")
    sentiment_df.to_csv(csv_path, index=False)
    print(f"✓ CSV saved to: {csv_path}")
    
    # 2. Excel with multiple sheets
    excel_path = os.path.join(OUT_TAB, "Sentiment_Analysis_Complete.xlsx")
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Full results
        sentiment_df.to_excel(writer, sheet_name='All_Results', index=False)
        
        # Summary by company
        summary_cols = ['positive_ratio', 'negative_ratio', 'uncertainty_ratio', 'net_sentiment']
        summary_by_company = sentiment_df.groupby('ticker')[summary_cols].agg(['mean', 'std', 'count']).round(4)
        summary_by_company.to_excel(writer, sheet_name='Company_Summary')
        
        # Key measures only
        key_cols = ['ticker', 'event_date', 'quarter', 'fiscal_year', 'positive_ratio', 
                   'negative_ratio', 'uncertainty_ratio', 'net_sentiment', 'sentiment_polarity']
        sentiment_df[key_cols].to_excel(writer, sheet_name='Key_Measures', index=False)
    
    print(f"✓ Excel saved to: {excel_path}")
    
    # 3. JSON format
    json_path = os.path.join(JSON_DIR, "sentiment_results.json")
    sentiment_json = sentiment_df.to_dict('records')
    with open(json_path, 'w') as f:
        json.dump(sentiment_json, f, indent=2, default=str)
    print(f"✓ JSON saved to: {json_path}")

# Perform sentiment analysis using transcripts from cell 1
if 'transcripts' in locals() and transcripts:
    print("[OK] Found transcripts variable, starting spaCy-based sentiment analysis...")
    sentiment_results = step3_sentiment_analysis_with_spacy(transcripts)
    
    # Store results with better column names for compatibility
    if sentiment_results is not None:
        # Create compatibility columns for subsequent cells
        sentiment_results['full_text_positive_ratio'] = sentiment_results['positive_ratio']
        sentiment_results['full_text_negative_ratio'] = sentiment_results['negative_ratio']
        sentiment_results['full_text_uncertainty_ratio'] = sentiment_results['uncertainty_ratio']
        sentiment_results['full_text_net_sentiment_ratio'] = sentiment_results['net_sentiment']
        sentiment_results['full_text_sentiment_polarity'] = sentiment_results['sentiment_polarity']
        
        print("[OK] Added compatibility columns for subsequent analysis steps")
else:
    print("[WARNING] Transcripts not found. Please run Step 1 first.")
    sentiment_results = None

C:\Users\ammaj\AppData\Roaming\Python\Python313\site-packages\cupy\_environment.py:215: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\ammaj\AppData\Roaming\Python\Python313\site-packages\cupy\_environment.py:215: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


✓ spaCy model loaded successfully
[OK] Found transcripts variable, starting spaCy-based sentiment analysis...
STEP 3: SENTIMENT ANALYSIS WITH SPACY
[OK] Using 4373 transcripts from memory
Loading Loughran-McDonald sentiment dictionary...
✓ Loading from Excel file: LoughranMcDonald_SentimentWordLists_2018.xlsx
✓ Converted to Python lists
✓ Processed with spaCy (lemmatized, lowercase)
✓ Converted to sets for fast word matching
✓ Final counts - Positive: 298, Negative: 1816, Uncertainty: 238

=== PROCESSING TRANSCRIPTS ONE BY ONE ===
✓ Processed 1/4373: UNH Q4
  Pos: 0.0378, Neg: 0.0239, Net: 0.0139
✓ Processed 2/4373: UNH Q3
  Pos: 0.0383, Neg: 0.0223, Net: 0.0160
✓ Processed 3/4373: UNH Q2
  Pos: 0.0357, Neg: 0.0305, Net: 0.0053
✓ Processed 4/4373: UNH Q1
  Pos: 0.0336, Neg: 0.0402, Net: -0.0066
✓ Processed 5/4373: UNH Q4
  Pos: 0.0478, Neg: 0.0252, Net: 0.0226
✓ Processed 50/4373: UNH Q1
  Pos: 0.0435, Neg: 0.0207, Net: 0.0229
✓ Processed 100/4373: MET Q2
  Pos: 0.0299, Neg: 0.0248, Ne

# Step 4: Merge CAR and Sentiment Data

**Input Files:**
- `CAR_DATA/CAR_-1_1_Results.csv` (from WRDS)
- `CAR_DATA/CAR_0_1_Results.csv` (from WRDS)
- `CAR_DATA/CAR_0_2_Results.csv` (from WRDS)
- `exports/tables/Sentiment_Analysis_Results.csv` (from Step 3)

**Output Files:**
- `exports/tables/CAR_Sentiment_Merged.csv`
This script merges CAR data from WRDS with sentiment analysis results.
Supports multiple CAR windows: 
### CAR(-1,1), CAR(0,1), CAR(0,2)

In [4]:
import pandas as pd
import os

# Configuration
OUT_TAB = "exports/tables"

def step4_merge_data():
    print("=" * 80)
    print("STEP 4: MERGING CAR AND SENTIMENT DATA")
    print("=" * 80)
    
    # Define CAR files to merge
    car_files = {
        'car_-1_1': "CAR_-1_1_Results.csv",
        'car_0_1': "CAR_0_1_Results.csv",
        'car_0_2': "CAR_0_2_Results.csv"
    }
    
    # Load sentiment data
    sentiment_path = os.path.join(OUT_TAB, "Sentiment_Analysis_Results.csv")
    if not os.path.exists(sentiment_path):
        print(" Sentiment data not found.")
        print("Please run 'python step3_sentiment_analysis.py' first.")
        return
    
    print("Loading sentiment data...")
    sentiment_df = pd.read_csv(sentiment_path)
    sentiment_df['event_date'] = pd.to_datetime(sentiment_df['event_date'])
    sentiment_df['event_date_only'] = sentiment_df['event_date'].dt.date
    print(f" Sentiment data: {len(sentiment_df)} records")
    
    # Remove duplicates from sentiment data
    sentiment_dups = sentiment_df.duplicated(subset=['ticker', 'event_date_only']).sum()
    if sentiment_dups > 0:
        print(f"  Found {sentiment_dups} sentiment duplicates - removing...")
        sentiment_df = sentiment_df.drop_duplicates(subset=['ticker', 'event_date_only'], keep='first')
    
    # Load and merge each CAR file
    car_data_dict = {}
    for car_name, car_file in car_files.items():
        car_path = os.path.join("CAR_DATA", car_file)
        
        if not os.path.exists(car_path):
            print(f"  {car_file} not found - skipping this CAR window")
            continue
        
        print(f"\nLoading {car_file}...")
        car_df = pd.read_csv(car_path)
        car_df['event_date'] = pd.to_datetime(car_df['event_date'])
        car_df['event_date_only'] = car_df['event_date'].dt.date
        print(f" {car_name.upper()}: {len(car_df)} records")
        
        # Remove duplicates
        car_dups = car_df.duplicated(subset=['ticker', 'event_date_only']).sum()
        if car_dups > 0:
            print(f"  Found {car_dups} duplicates - removing...")
            car_df = car_df.drop_duplicates(subset=['ticker', 'event_date_only'], keep='first')
        
        # Select relevant columns and rename with CAR window suffix
        car_columns = ['ticker', 'event_date_only', 'car', 'car_tstat', 
                      'alpha', 'beta', 'r_squared', 'observations']
        car_df_subset = car_df[car_columns].copy()
        
        # Rename columns to include CAR window
        rename_dict = {
            'car': car_name,
            'car_tstat': f'{car_name}_tstat',
            'alpha': f'{car_name}_alpha',
            'beta': f'{car_name}_beta',
            'r_squared': f'{car_name}_rsquared',
            'observations': f'{car_name}_obs'
        }
        car_df_subset = car_df_subset.rename(columns=rename_dict)
        
        car_data_dict[car_name] = car_df_subset
    
    if not car_data_dict:
        print(" No CAR data files found!")
        print("Please run WRDS analysis first to generate CAR files.")
        return
    
    # Start with sentiment data
    merged_df = sentiment_df.copy()
    
    # Merge each CAR dataset
    for car_name, car_df in car_data_dict.items():
        print(f"\nMerging {car_name.upper()}...")
        
        merged_df = pd.merge(
            merged_df,
            car_df,
            on=['ticker', 'event_date_only'],
            how='left'
        )
        
        matched = merged_df[car_name].notna().sum()
        print(f" Matched {matched} events ({matched/len(merged_df)*100:.1f}%)")
    
    # Remove rows where ALL CAR values are missing
    car_cols = [col for col in merged_df.columns if col.startswith('car_')]
    merged_df = merged_df.dropna(subset=car_cols, how='all')
    
    print(f"\n Final merged dataset: {len(merged_df)} records")
    print(f"   Companies: {merged_df['ticker'].nunique()}")
    
    # Save merged dataset
    merged_path = os.path.join(OUT_TAB, "CAR_Sentiment_Merged.csv")
    merged_df.to_csv(merged_path, index=False)
    
    # Save Excel version with multiple sheets
    excel_path = os.path.join(OUT_TAB, "CAR_Sentiment_Analysis.xlsx")
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Main dataset
        merged_df.to_excel(writer, sheet_name='Merged_Data', index=False)
        
        # Summary statistics
        numeric_cols = merged_df.select_dtypes(include=['number']).columns
        summary_stats = merged_df[numeric_cols].describe()
        summary_stats.to_excel(writer, sheet_name='Summary_Statistics')
        
        # CAR comparison by company
        car_comparison_cols = ['ticker'] + [col for col in merged_df.columns if col.startswith('car_') and not any(x in col for x in ['tstat', 'alpha', 'beta', 'rsquared', 'obs'])]
        if len(car_comparison_cols) > 1:
            car_by_company = merged_df.groupby('ticker')[car_comparison_cols[1:]].agg(['mean', 'std', 'count']).round(4)
            car_by_company.to_excel(writer, sheet_name='CAR_by_Company')
        
        # Sentiment summary
        key_sentiment_cols = [col for col in merged_df.columns if 'sentiment' in col.lower() and 'ratio' in col.lower()]
        if key_sentiment_cols:
            sentiment_summary = merged_df.groupby('ticker')[key_sentiment_cols].mean().round(4)
            sentiment_summary.to_excel(writer, sheet_name='Sentiment_by_Company')
    
    print("\n Files saved:")
    print(f"   - {merged_path}")
    print(f"   - {excel_path}")
    
    # Display correlations for each CAR window
    print("\n CORRELATIONS WITH SENTIMENT:")
    sentiment_cols = ['negative_ratio', 'positive_ratio', 'uncertainty_ratio']
    
    for car_col in [col for col in merged_df.columns if col.startswith('car_') and col.count('_') == 2]:
        print(f"\n{car_col.upper()}:")
        for sent_col in sentiment_cols:
            if sent_col in merged_df.columns and car_col in merged_df.columns:
                corr = merged_df[[car_col, sent_col]].corr().iloc[0,1]
                print(f"  {sent_col}: {corr:.4f}")
    
    return merged_df

step4_merge_data()

STEP 4: MERGING CAR AND SENTIMENT DATA
Loading sentiment data...
 Sentiment data: 4373 records
  Found 25 sentiment duplicates - removing...

Loading CAR_-1_1_Results.csv...
 CAR_-1_1: 4373 records
  Found 25 duplicates - removing...

Loading CAR_0_1_Results.csv...
 CAR_0_1: 4348 records

Loading CAR_0_2_Results.csv...
 CAR_0_2: 4348 records

Merging CAR_-1_1...
 Matched 4166 events (95.8%)

Merging CAR_0_1...
 Matched 3360 events (77.3%)

Merging CAR_0_2...
 Matched 3390 events (78.0%)

 Final merged dataset: 4348 records
   Companies: 60

 Files saved:
   - exports/tables\CAR_Sentiment_Merged.csv
   - exports/tables\CAR_Sentiment_Analysis.xlsx

 CORRELATIONS WITH SENTIMENT:

CAR_-1_1:
  negative_ratio: -0.0346
  positive_ratio: 0.0196
  uncertainty_ratio: 0.0214

CAR_0_1:
  negative_ratio: -0.0330
  positive_ratio: 0.0188
  uncertainty_ratio: 0.0264

CAR_0_2:
  negative_ratio: -0.0296
  positive_ratio: 0.0199
  uncertainty_ratio: 0.0265


,ticker,quarter,fiscal_year,event_date,transcript_id,total_words,positive_count,negative_count,uncertainty_count,positive_ratio,...,car_0_1_alpha,car_0_1_beta,car_0_1_rsquared,car_0_1_obs,car_0_2,car_0_2_tstat,car_0_2_alpha,car_0_2_beta,car_0_2_rsquared,car_0_2_obs
0,ABT,2,2007,2007-07-18 19:16:50,ABT_2330,4580,122,82,71,0.026638,...,0.001000,0.645563,0.144958,2,-0.006924,-0.258297,0.001000,0.645563,0.144958,3
1,ABT,3,2007,2007-10-17 17:53:08,ABT_3153,4412,126,73,51,0.028558,...,-0.000069,0.800778,0.334748,2,0.006401,0.219363,-0.000069,0.800778,0.334748,3
2,ABT,4,2007,2008-01-23 21:45:06,ABT_4880,4882,167,75,74,0.034207,...,-0.000030,0.679371,0.366176,2,-0.048132,-2.812429,-0.000030,0.679371,0.366176,3
3,ABT,1,2008,2008-04-16 18:24:19,ABT_7370,4680,131,76,70,0.027991,...,0.000152,0.500453,0.247415,2,-0.043338,-3.197722,0.000152,0.500453,0.247415,3
4,ABT,2,2008,2008-07-17 00:49:23,ABT_10085,5096,184,70,70,0.036107,...,0.000488,0.421022,0.158410,2,0.005516,1.386714,0.000488,0.421022,0.158410,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4343,XOM,1,2020,2020-05-01 20:00:40,XOM_167742,1616,62,59,22,0.038366,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0
4344,XOM,0,2020,2020-05-27 21:59:15,XOM_170399,5579,168,192,68,0.030113,...,-0.002662,1.050242,0.709236,2,-0.018166,-0.779919,-0.002662,1.050242,0.709236,3
4345,XOM,2,2020,2020-07-31 17:59:08,XOM_172110,5973,216,200,88,0.036163,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0
4346,XOM,3,2020,2020-10-30 21:24:08,XOM_176630,5304,181,156,61,0.034125,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0


# Step 5: Collect Market Cap and Volume Data

**Input Files:**
- `exports/tables/CAR_Sentiment_Merged.csv` (from Step 4)
- Yahoo Finance API (online data source)

**Output Files:**
- `exports/tables/Market_Cap_Daily_Data.csv`
- `exports/tables/Market_Cap_Monthly_Lookup.csv`
- `exports/tables/Market_Cap_Complete_Data.xlsx`
- `exports/tables/loading_summary.csv`
This script collects real market capitalization data from Yahoo Finance
for all companies in our dataset and saves it for use in the analysis.

In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# Configuration
OUTPUT_DIR = "exports/tables"

def get_companies_from_dataset(output_dir: str = OUTPUT_DIR) -> list:
    """Get list of companies from the merged dataset"""
    merged_path = os.path.join(output_dir, "CAR_Sentiment_Merged.csv")
    
    if not os.path.exists(merged_path):
        raise FileNotFoundError(f"Merged dataset not found: {merged_path}")
    
    df = pd.read_csv(merged_path)
    companies = sorted(df['ticker'].unique().tolist())
    
    print(f" Found {len(companies)} companies in dataset:")
    print(f"Companies: {', '.join(companies)}")
    
    return companies

def get_date_range_from_dataset(output_dir: str = OUTPUT_DIR) -> tuple:
    """Get the date range from our dataset"""
    merged_path = os.path.join(output_dir, "CAR_Sentiment_Merged.csv")
    df = pd.read_csv(merged_path)
    df['event_date'] = pd.to_datetime(df['event_date'])
    
    start_date = df['event_date'].min()
    end_date = df['event_date'].max()
    
    # Extend range slightly to ensure we have data
    start_date = start_date - timedelta(days=365)
    end_date = end_date + timedelta(days=365)
    
    print(f" Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    
    return start_date, end_date

def collect_single_company_data(ticker: str, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    """Collect market cap data for a single company"""
    try:
        print(f"Collecting data for {ticker}...")
        
        # Create ticker object
        stock = yf.Ticker(ticker)
        
        # Get historical data (daily)
        hist_data = stock.history(start=start_date, end=end_date, interval='1d')
        
        if hist_data.empty:
            print(f" No historical data found for {ticker}")
            return pd.DataFrame()
        
        # Get current info for shares outstanding
        try:
            info = stock.info
            shares_outstanding = info.get('sharesOutstanding', None)
            
            # If shares outstanding not available, try other methods
            if shares_outstanding is None:
                shares_outstanding = info.get('impliedSharesOutstanding', None)
            if shares_outstanding is None:
                shares_outstanding = info.get('floatShares', None)
                
        except Exception as e:
            print(f" Could not get shares outstanding for {ticker}: {e}")
            shares_outstanding = None
        
        # Calculate market cap
        if shares_outstanding is not None:
            hist_data['market_cap'] = hist_data['Close'] * shares_outstanding
        else:
            # Estimate shares outstanding from recent market cap if available
            try:
                recent_market_cap = info.get('marketCap', None)
                if recent_market_cap is not None and not hist_data.empty:
                    # Use most recent close price to estimate shares
                    recent_price = hist_data['Close'].iloc[-1]
                    estimated_shares = recent_market_cap / recent_price
                    hist_data['market_cap'] = hist_data['Close'] * estimated_shares
                    print(f"   Estimated shares outstanding for {ticker}: {estimated_shares:,.0f}")
                else:
                    hist_data['market_cap'] = np.nan
            except:
                hist_data['market_cap'] = np.nan
        
        # Add ticker column
        hist_data['ticker'] = ticker
        hist_data['date'] = hist_data.index.date
        
        # Keep only relevant columns (including Volume for turnover analysis)
        result = hist_data[['ticker', 'date', 'Close', 'Volume', 'market_cap']].copy()
        result.columns = ['ticker', 'date', 'stock_price', 'volume', 'market_cap']
        result.reset_index(drop=True, inplace=True)
        
        print(f"  Collected {len(result)} records for {ticker}")
        
        # Add small delay to be respectful to Yahoo Finance
        time.sleep(0.5)
        
        return result
        
    except Exception as e:
        print(f" Error collecting data for {ticker}: {e}")
        return pd.DataFrame()

def collect_all_market_cap_data() -> pd.DataFrame:
    """Collect market cap data for all companies"""
    print("=" * 80)
    print("COLLECTING MARKET CAPITALIZATION DATA FROM YAHOO FINANCE")
    print("=" * 80)
    
    # Get companies and date range
    companies = get_companies_from_dataset()
    start_date, end_date = get_date_range_from_dataset()
    
    all_data = []
    
    # Collect data for each company
    for i, ticker in enumerate(companies, 1):
        print(f"\n[{i}/{len(companies)}] Processing {ticker}...")
        
        company_data = collect_single_company_data(ticker, start_date, end_date)
        
        if not company_data.empty:
            all_data.append(company_data)
        
        # Progress update
        if i % 10 == 0:
            print(f"\n Progress: {i}/{len(companies)} companies completed")
    
    # Combine all data
    if all_data:
        combined_data = pd.concat(all_data, ignore_index=True)
        combined_data['date'] = pd.to_datetime(combined_data['date'])
        
        print(f"\n Successfully collected data for {len(all_data)} companies")
        print(f"Total records: {len(combined_data)}")
        
        return combined_data
    else:
        print(" No data collected")
        return pd.DataFrame()

def clean_and_validate_data(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and validate the collected market cap data"""
    print("\nCleaning and validating market cap data...")
    
    if df.empty:
        return df
    
    original_count = len(df)
    
    # Remove rows with missing market cap
    df = df.dropna(subset=['market_cap'])
    
    # Remove negative or zero market caps
    df = df[df['market_cap'] > 0]
    
    # Remove extreme outliers (market cap > $10 trillion - likely data errors)
    df = df[df['market_cap'] < 10e12]
    
    # Sort by ticker and date
    df = df.sort_values(['ticker', 'date'])
    
    print(f" Data cleaned: {original_count} → {len(df)} records")
    print(f" Companies with data: {df['ticker'].nunique()}")
    print(f" Date range: {df['date'].min()} to {df['date'].max()}")
    
    return df

def create_market_cap_lookup(df: pd.DataFrame) -> pd.DataFrame:
    """Create a lookup table for market cap by ticker and date"""
    if df.empty:
        return df
    
    print("\nCreating market cap lookup table...")
    
    # Create monthly averages to reduce data size
    df['year_month'] = df['date'].dt.to_period('M')
    
    monthly_data = df.groupby(['ticker', 'year_month']).agg({
        'market_cap': 'mean',
        'stock_price': 'mean',
        'date': 'first'
    }).reset_index()
    
    # Convert back to datetime
    monthly_data['date'] = monthly_data['year_month'].dt.start_time
    monthly_data = monthly_data.drop('year_month', axis=1)
    
    # Add log market cap
    monthly_data['log_market_cap'] = np.log(monthly_data['market_cap'])
    
    print(f" Created monthly lookup table: {len(monthly_data)} records")
    
    return monthly_data

def save_market_cap_data(df: pd.DataFrame, lookup_df: pd.DataFrame, output_dir: str = OUTPUT_DIR) -> str:
    """Save the market cap data"""
    
    # Save daily data
    daily_path = os.path.join(output_dir, "Market_Cap_Daily_Data.csv")
    df.to_csv(daily_path, index=False)
    
    # Save monthly lookup
    monthly_path = os.path.join(output_dir, "Market_Cap_Monthly_Lookup.csv")
    lookup_df.to_csv(monthly_path, index=False)
    
    # Save Excel with summary
    excel_path = os.path.join(output_dir, "Market_Cap_Complete_Data.xlsx")
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Monthly lookup (main table for merging)
        lookup_df.to_excel(writer, sheet_name='Monthly_Lookup', index=False)
        
        # Daily data (complete)
        if len(df) < 50000:  # Only if not too large
            df.to_excel(writer, sheet_name='Daily_Data', index=False)
        
        # Summary statistics
        summary_stats = lookup_df.groupby('ticker').agg({
            'market_cap': ['mean', 'std', 'min', 'max', 'count'],
            'log_market_cap': ['mean', 'std'],
            'date': ['min', 'max']
        }).round(2)
        summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns]
        summary_stats.to_excel(writer, sheet_name='Summary_by_Company')
        
        # Overall summary
        overall_summary = pd.DataFrame({
            'Metric': ['Total Companies', 'Total Records', 'Date Range Start', 'Date Range End',
                      'Mean Market Cap ($B)', 'Median Market Cap ($B)', 'Max Market Cap ($B)'],
            'Value': [lookup_df['ticker'].nunique(), len(lookup_df), 
                     lookup_df['date'].min(), lookup_df['date'].max(),
                     f"{lookup_df['market_cap'].mean()/1e9:.1f}", 
                     f"{lookup_df['market_cap'].median()/1e9:.1f}",
                     f"{lookup_df['market_cap'].max()/1e9:.1f}"]
        })
        overall_summary.to_excel(writer, sheet_name='Overall_Summary', index=False)
    
    print(f"\n Market cap data saved:")
    print(f"  - {daily_path}")
    print(f"  - {monthly_path}")
    print(f"  - {excel_path}")
    
    return monthly_path

def step5_collect_market_cap_data():
    """Main function to collect market cap data"""
    
    print("=" * 80)
    print("MARKET CAPITALIZATION DATA COLLECTION")
    print("=" * 80)
    
    try:
        # Collect all market cap data
        market_cap_data = collect_all_market_cap_data()
        
        if not market_cap_data.empty:
            # Clean and validate
            clean_data = clean_and_validate_data(market_cap_data)
            
            # Create lookup table
            lookup_data = create_market_cap_lookup(clean_data)
            
            # Save data
            lookup_path = save_market_cap_data(clean_data, lookup_data)
            
            
            print("\n" + "=" * 80)
            print(" MARKET CAP DATA COLLECTION COMPLETED!")
            print("=" * 80)
            print(f"Use this file for merging: {lookup_path}")
            print("Ready to update the control variables script!")
            
        else:
            print(" No market cap data collected")
            
    except Exception as e:
        print(f" Error: {str(e)}")
        import traceback
        traceback.print_exc()


step5_collect_market_cap_data()

MARKET CAPITALIZATION DATA COLLECTION
COLLECTING MARKET CAPITALIZATION DATA FROM YAHOO FINANCE
 Found 60 companies in dataset:
Companies: ABT, ADBE, AMGN, AMT, ASML, AXP, AZN, BA, BAC, BKNG, BLK, BMY, CMCSA, COST, CRM, DUK, EBAY, F, FB, FDX, GM, GOOG, GS, HD, HON, IBM, INTC, INTU, ISRG, JNJ, JPM, LLY, LMT, MA, MCD, MCO, MDT, MET, MS, MSFT, NFLX, NKE, ORCL, PEP, PG, PM, QCOM, SAP, SNY, T, TD, TMO, TMUS, TSLA, TSM, UNH, UNP, UPS, V, XOM
 Date range: 2004-10-31 to 2022-03-12

[1/60] Processing ABT...
  Collected 4371 records for ABT

[2/60] Processing ADBE...
  Collected 4371 records for ADBE

[3/60] Processing AMGN...
  Collected 4371 records for AMGN

[4/60] Processing AMT...
  Collected 4371 records for AMT

[5/60] Processing ASML...
  Collected 4371 records for ASML

[6/60] Processing AXP...
  Collected 4371 records for AXP

[7/60] Processing AZN...
  Collected 4371 records for AZN

[8/60] Processing BA...
  Collected 4371 records for BA

[9/60] Processing BAC...
  Collected 4371 reco

$FB: possibly delisted; no price data found  (1d 2004-10-31 00:40:28 -> 2022-03-12 01:38:07) (Yahoo error = "Data doesn't exist for startDate = 1099197628, endDate = 1647067087")


 No historical data found for FB

[20/60] Processing FDX...
  Collected 4371 records for FDX

 Progress: 20/60 companies completed

[21/60] Processing GM...
  Collected 2847 records for GM

[22/60] Processing GOOG...
  Collected 4371 records for GOOG

[23/60] Processing GS...
  Collected 4371 records for GS

[24/60] Processing HD...
  Collected 4371 records for HD

[25/60] Processing HON...
  Collected 4371 records for HON

[26/60] Processing IBM...
  Collected 4371 records for IBM

[27/60] Processing INTC...
  Collected 4371 records for INTC

[28/60] Processing INTU...
  Collected 4371 records for INTU

[29/60] Processing ISRG...
  Collected 4371 records for ISRG

[30/60] Processing JNJ...
  Collected 4371 records for JNJ

 Progress: 30/60 companies completed

[31/60] Processing JPM...
  Collected 4371 records for JPM

[32/60] Processing LLY...
  Collected 4371 records for LLY

[33/60] Processing LMT...
  Collected 4371 records for LMT

[34/60] Processing MA...
  Collected 3977 record

# Step 6: Add Control Variables

**Input Files:**
- `exports/tables/CAR_Sentiment_Merged.csv` (from Step 4)
- `exports/tables/Market_Cap_Daily_Data.csv` (from Step 5)

**Output Files:**
- `exports/tables/Final_Regression_Dataset.csv`

**Control Variables Added:**
- Firm size (log market cap)
- Volatility (price volatility, systematic risk)
- Temporal (year, quarter, covid period)
- Performance (previous CAR)
- Sentiment complexity (word count, ratios)
- Stock price momentum (1 week, 2 week, 1 month)
- Abnormal turnover (volume analysis)
- Industry dummies

Updated Control Variables Module - Using Real Market Cap Data
This module adds control variables using REAL market capitalization data
collected from Yahoo Finance instead of estimates.
Supports multiple CAR windows: CAR(-1,1), CAR(0,1), CAR(0,2)

In [6]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

def load_merged_data(output_dir: str = "exports/tables") -> pd.DataFrame:
    """Load the merged CAR-sentiment dataset"""
    merged_path = os.path.join(output_dir, "CAR_Sentiment_Merged.csv")
    
    if not os.path.exists(merged_path):
        raise FileNotFoundError(f"Merged dataset not found: {merged_path}")
    
    df = pd.read_csv(merged_path)
    df['event_date'] = pd.to_datetime(df['event_date'])
    
    # Add backward compatibility aliases for regression analysis
    df['full_text_positive_ratio'] = df['positive_ratio']
    df['full_text_negative_ratio'] = df['negative_ratio']
    df['full_text_uncertainty_ratio'] = df['uncertainty_ratio']
    df['full_text_sentiment_polarity'] = df['sentiment_polarity']
    df['full_text_total_words'] = df['total_words']
    
    # Create 'car' as primary CAR measure (use CAR(-1,1) if available, otherwise first available)
    if 'car_-1_1' in df.columns:
        df['car'] = df['car_-1_1']
        df['car_tstat'] = df['car_-1_1_tstat']
        df['alpha'] = df['car_-1_1_alpha']
        df['beta'] = df['car_-1_1_beta']
        df['r_squared'] = df['car_-1_1_rsquared']
        print(" Using CAR(-1,1) as primary CAR measure")
    elif 'car_0_1' in df.columns:
        df['car'] = df['car_0_1']
        df['car_tstat'] = df['car_0_1_tstat']
        df['alpha'] = df['car_0_1_alpha']
        df['beta'] = df['car_0_1_beta']
        df['r_squared'] = df['car_0_1_rsquared']
        print(" Using CAR(0,1) as primary CAR measure")
    elif 'car_0_2' in df.columns:
        df['car'] = df['car_0_2']
        df['car_tstat'] = df['car_0_2_tstat']
        df['alpha'] = df['car_0_2_alpha']
        df['beta'] = df['car_0_2_beta']
        df['r_squared'] = df['car_0_2_rsquared']
        print(" Using CAR(0,2) as primary CAR measure")
    
    print(f" Loaded merged dataset: {len(df)} records")
    
    # Show available CAR windows
    car_windows = [col for col in df.columns if col.startswith('car_') and col.count('_') == 2]
    if car_windows:
        print(f" Available CAR windows: {', '.join([c.upper() for c in car_windows])}")
    
    return df

def load_real_market_cap_data(output_dir: str = "exports/tables") -> pd.DataFrame:
    """Load the real market cap data from Yahoo Finance"""
    market_cap_path = os.path.join(output_dir, "Market_Cap_Monthly_Lookup.csv")
    
    if not os.path.exists(market_cap_path):
        raise FileNotFoundError(f"Market cap data not found: {market_cap_path}")
    
    df = pd.read_csv(market_cap_path)
    df['date'] = pd.to_datetime(df['date'])
    
    print(f" Loaded real market cap data: {len(df)} records for {df['ticker'].nunique()} companies")
    print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
    
    return df

def find_closest_market_cap(ticker: str, event_date: pd.Timestamp, market_cap_data: pd.DataFrame) -> dict:
    """Find the closest market cap data for a given ticker and date"""
    if market_cap_data is None or market_cap_data.empty:
        return {'market_cap': np.nan, 'log_market_cap': np.nan, 'stock_price': np.nan}
    
    # Filter data for this ticker
    ticker_data = market_cap_data[market_cap_data['ticker'] == ticker].copy()
    
    if ticker_data.empty:
        return {'market_cap': np.nan, 'log_market_cap': np.nan, 'stock_price': np.nan}
    
    # Calculate time differences
    ticker_data['time_diff'] = abs((ticker_data['date'] - event_date).dt.days)
    
    # Find the closest date (within 60 days)
    closest_row = ticker_data[ticker_data['time_diff'] <= 60].nsmallest(1, 'time_diff')
    
    if closest_row.empty:
        return {'market_cap': np.nan, 'log_market_cap': np.nan, 'stock_price': np.nan}
    
    row = closest_row.iloc[0]
    return {
        'market_cap': row['market_cap'],
        'log_market_cap': row['log_market_cap'],
        'stock_price': row['stock_price'],
        'market_cap_date': row['date'],
        'days_diff': row['time_diff']
    }

def add_real_firm_size_control(df: pd.DataFrame, output_dir: str = "exports/tables") -> pd.DataFrame:
    """
    Add firm size control variable using REAL Market Capitalization from Yahoo Finance
    """
    print("Adding firm size control variable (REAL Market Capitalization from Yahoo Finance)...")
    
    df = df.copy()
    
    # Load real market cap data
    market_cap_data = load_real_market_cap_data(output_dir)
    
    # Initialize new columns
    df['market_cap'] = np.nan
    df['log_market_cap'] = np.nan
    df['stock_price'] = np.nan
    df['market_cap_date'] = pd.NaT
    df['market_cap_days_diff'] = np.nan
    
    # Match market cap for each observation
    print("Matching market cap data to earnings events...")
    for idx, row in df.iterrows():
        market_cap_info = find_closest_market_cap(row['ticker'], row['event_date'], market_cap_data)
        
        df.at[idx, 'market_cap'] = market_cap_info['market_cap']
        df.at[idx, 'log_market_cap'] = market_cap_info['log_market_cap']
        df.at[idx, 'stock_price'] = market_cap_info['stock_price']
        
        if 'market_cap_date' in market_cap_info:
            df.at[idx, 'market_cap_date'] = market_cap_info['market_cap_date']
            df.at[idx, 'market_cap_days_diff'] = market_cap_info['days_diff']
        
        # Progress indicator
        if (idx + 1) % 500 == 0:
            print(f"  Processed {idx + 1}/{len(df)} records...")
    
    # Remove rows where we couldn't find market cap data
    original_count = len(df)
    df = df.dropna(subset=['market_cap'])
    
    print(f" Market cap matching completed: {original_count} → {len(df)} records")
        
    # Create additional size measures
    print("Creating additional firm size measures...")
        
    # Size quintiles based on real market cap
    df['market_cap_quintile'] = pd.qcut(df['market_cap'], 
                                          q=5, 
                                          labels=['Small', 'Small-Mid', 'Mid', 'Mid-Large', 'Large'])
        
    # Size ranks (0-1 scale)
    df['firm_size_rank'] = df['market_cap'].rank(pct=True)
        
    # Binary large vs small indicator (top 30%)
    df['large_firm'] = (df['firm_size_rank'] > 0.7).astype(int)
        
    # Market cap in billions for easier interpretation
    df['market_cap_billions'] = df['market_cap'] / 1e9
        
    # Size categories based on common finance thresholds
    df['size_category'] = pd.cut(df['market_cap_billions'], 
                                   bins=[0, 2, 10, 50, 200, float('inf')],
                                   labels=['Micro', 'Small', 'Mid', 'Large', 'Mega'])
        
    print(" Added REAL firm size controls: market_cap, log_market_cap, market_cap_quintile, firm_size_rank, large_firm")
        
    # Print summary statistics
    print(f"\nMarket Cap Summary ($ Billions):")
    print(f"  Mean: ${df['market_cap_billions'].mean():.1f}B")
    print(f"  Median: ${df['market_cap_billions'].median():.1f}B")
    print(f"  Min: ${df['market_cap_billions'].min():.1f}B")
    print(f"  Max: ${df['market_cap_billions'].max():.1f}B")
        
    return df

def add_volatility_control(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add stock price volatility control (required by assignment)
    Using CAR t-statistic and market model R-squared as volatility proxies
    """
    print("Adding volatility control variables...")
    
    df = df.copy()
    
    # Volatility proxy 1: Absolute CAR t-statistic (using primary CAR)
    if 'car_tstat' in df.columns:
        df['volatility_tstat'] = np.abs(df['car_tstat'])
    
    # Volatility proxy 2: Inverse of market model R-squared (lower R² = higher idiosyncratic risk)
    if 'r_squared' in df.columns:
        df['volatility_rsquared'] = 1 - df['r_squared']
    
    # Volatility proxy 3: Beta-based systematic risk
    if 'beta' in df.columns:
        df['systematic_risk'] = np.abs(df['beta'])
    
    # Combined volatility measure
    if 'volatility_tstat' in df.columns and 'volatility_rsquared' in df.columns:
        df['volatility_combined'] = (df['volatility_tstat'] + df['volatility_rsquared']) / 2
    
    print(" Added volatility controls: volatility_tstat, volatility_rsquared, systematic_risk")
    return df

def add_temporal_controls(df: pd.DataFrame) -> pd.DataFrame:
    """Add time-based control variables"""
    print("Adding temporal control variables...")
    
    df = df.copy()
    
    # Year and quarter effects
    df['year'] = df['event_date'].dt.year
    if 'quarter' not in df.columns:
        df['quarter'] = df['event_date'].dt.quarter
    df['quarter'] = df['quarter'].astype(int)
    
    # Financial crisis indicator (2008-2009)
    df['financial_crisis'] = ((df['year'] >= 2008) & (df['year'] <= 2009)).astype(int)
    
    # COVID period indicator (2020-2021)
    df['covid_period'] = ((df['year'] >= 2020) & (df['year'] <= 2021)).astype(int)
    
    # Time trend
    min_year = df['year'].min()
    df['time_trend'] = df['year'] - min_year
    
    print(" Added temporal controls: year, quarter, financial_crisis, covid_period, time_trend")
    return df

def add_performance_controls(df: pd.DataFrame) -> pd.DataFrame:
    """Add past performance and market condition controls"""
    print("Adding performance control variables...")
    
    df = df.copy()
    
    # Sort by ticker and date for lagged variables
    df = df.sort_values(['ticker', 'event_date'])
    
    # Past performance proxy: Lagged CAR (previous quarter's performance) - using primary CAR
    if 'car' in df.columns:
        df['past_car'] = df.groupby('ticker')['car'].shift(1)
        
        # Performance consistency: Rolling standard deviation of CAR
        df['car_consistency'] = df.groupby('ticker')['car'].rolling(window=4, min_periods=2).std().reset_index(0, drop=True)
    
    # Market model quality (higher R² = better market fit)
    if 'r_squared' in df.columns:
        df['market_model_quality'] = df['r_squared']
    
    # Alpha significance (measure of abnormal performance)
    if 'alpha' in df.columns:
        df['alpha_magnitude'] = np.abs(df['alpha'])
    
    print(" Added performance controls: past_car, car_consistency, market_model_quality, alpha_magnitude")
    return df

def add_sentiment_complexity_controls(df: pd.DataFrame) -> pd.DataFrame:
    """Add additional sentiment-based controls"""
    print("Adding sentiment complexity controls...")
    
    df = df.copy()
    
    # Text length as information complexity proxy
    df['transcript_length'] = df['total_words']
    df['log_transcript_length'] = np.log(df['transcript_length'] + 1)
    
    # Sentiment balance measures
    df['sentiment_balance'] = np.abs(df['positive_ratio'] - df['negative_ratio'])
    
    # Net sentiment strength (absolute value of net sentiment)
    df['sentiment_strength'] = np.abs(df['net_sentiment'])
    
    # Uncertainty intensity (using available uncertainty ratio)
    df['uncertainty_intensity'] = df['uncertainty_ratio']
    
    # Sentiment concentration (how concentrated is the sentiment)
    df['sentiment_concentration'] = df['positive_ratio'] + df['negative_ratio']
    
    print(" Added sentiment controls: transcript_length, sentiment_balance, sentiment_strength, uncertainty_intensity")
    return df

def add_stock_price_momentum_controls(df: pd.DataFrame, output_dir: str = "exports/tables") -> pd.DataFrame:
    """
    Add stock price momentum controls to measure pre-event price trends
    
    This captures whether the stock was trending up or down before the earnings event,
    which could reduce the impact of sentiment (momentum continuation hypothesis)
    """
    print("Adding stock price momentum controls...")
    
    df = df.copy()
    
    # Load daily stock price data
    daily_price_path = os.path.join(output_dir, "Market_Cap_Daily_Data.csv")
    
    if not os.path.exists(daily_price_path):
        print("  Daily price data not found. Skipping momentum controls.")
        return df
    
    daily_data = pd.read_csv(daily_price_path)
    daily_data['date'] = pd.to_datetime(daily_data['date'])
    
    # Initialize momentum columns
    df['price_momentum_1week'] = np.nan
    df['price_momentum_2week'] = np.nan
    df['price_momentum_1month'] = np.nan
    df['price_trend'] = np.nan  # Binary: 1 = uptrend, 0 = downtrend
    df['price_volatility_pre_event'] = np.nan
    df['cumulative_return_pre_event'] = np.nan
    
    print("Calculating pre-event price momentum for each observation...")
    
    for idx, row in df.iterrows():
        ticker = row['ticker']
        event_date = row['event_date']
        
        # Get stock prices for this company
        ticker_prices = daily_data[daily_data['ticker'] == ticker].copy()
        ticker_prices = ticker_prices.sort_values('date')
        
        # Filter prices before the event
        pre_event_prices = ticker_prices[ticker_prices['date'] < event_date].copy()
        
        if len(pre_event_prices) < 5:
            continue  # Not enough data
        
        # Get the most recent price before event (reference price)
        reference_price = pre_event_prices.iloc[-1]['stock_price']
        reference_date = pre_event_prices.iloc[-1]['date']
        
        # Calculate 1-week momentum (5 trading days before event)
        days_back_1week = 5
        if len(pre_event_prices) >= days_back_1week + 1:
            price_1week_ago = pre_event_prices.iloc[-(days_back_1week + 1)]['stock_price']
            if price_1week_ago > 0:
                momentum_1week = (reference_price - price_1week_ago) / price_1week_ago
                df.at[idx, 'price_momentum_1week'] = momentum_1week
        
        # Calculate 2-week momentum (10 trading days before event)
        days_back_2week = 10
        if len(pre_event_prices) >= days_back_2week + 1:
            price_2week_ago = pre_event_prices.iloc[-(days_back_2week + 1)]['stock_price']
            if price_2week_ago > 0:
                momentum_2week = (reference_price - price_2week_ago) / price_2week_ago
                df.at[idx, 'price_momentum_2week'] = momentum_2week
        
        # Calculate 1-month momentum (20 trading days before event)
        days_back_1month = 20
        if len(pre_event_prices) >= days_back_1month + 1:
            price_1month_ago = pre_event_prices.iloc[-(days_back_1month + 1)]['stock_price']
            if price_1month_ago > 0:
                momentum_1month = (reference_price - price_1month_ago) / price_1month_ago
                df.at[idx, 'price_momentum_1month'] = momentum_1month
                
                # Determine price trend (1 = uptrend, 0 = downtrend)
                df.at[idx, 'price_trend'] = 1 if momentum_1month > 0 else 0
        
        # Calculate pre-event volatility (standard deviation of returns in last 20 days)
        if len(pre_event_prices) >= 21:
            recent_prices = pre_event_prices.iloc[-21:]['stock_price'].values
            returns = np.diff(recent_prices) / recent_prices[:-1]
            price_volatility = np.std(returns)
            df.at[idx, 'price_volatility_pre_event'] = price_volatility
        
        # Calculate cumulative return over the pre-event period (last 20 days)
        if len(pre_event_prices) >= 21:
            price_20days_ago = pre_event_prices.iloc[-21]['stock_price']
            if price_20days_ago > 0:
                cumulative_return = (reference_price - price_20days_ago) / price_20days_ago
                df.at[idx, 'cumulative_return_pre_event'] = cumulative_return
        
        # Progress indicator
        if (idx + 1) % 500 == 0:
            print(f"  Processed {idx + 1}/{len(df)} records...")
    
    # Create interaction terms (for regression analysis)
    # These capture how sentiment effect varies with momentum
    
    if 'price_momentum_1month' in df.columns and 'positive_ratio' in df.columns:
        # Positive sentiment has less effect when stock already rising
        df['momentum_x_positive_sent'] = df['price_momentum_1month'] * df['positive_ratio']
        
        # Negative sentiment has less effect when stock already falling
        df['momentum_x_negative_sent'] = df['price_momentum_1month'] * df['negative_ratio']
    
    # Create momentum categories for easier interpretation
    if 'price_momentum_1month' in df.columns:
        df['momentum_category'] = pd.cut(
            df['price_momentum_1month'],
            bins=[-np.inf, -0.10, -0.02, 0.02, 0.10, np.inf],
            labels=['Strong Down', 'Weak Down', 'Flat', 'Weak Up', 'Strong Up']
        )
    
    print(" Added momentum controls:")
    print("   - price_momentum_1week, price_momentum_2week, price_momentum_1month")
    print("   - price_trend (binary), price_volatility_pre_event")
    print("   - cumulative_return_pre_event, momentum_category")
    print("   - momentum_x_positive_sent, momentum_x_negative_sent (interaction terms)")
    
    # Print summary statistics
    if 'price_momentum_1month' in df.columns:
        valid_momentum = df['price_momentum_1month'].dropna()
        if len(valid_momentum) > 0:
            print(f"\n Pre-Event Momentum Summary (1-month):")
            print(f"   Mean: {valid_momentum.mean():.4f} ({valid_momentum.mean()*100:.2f}%)")
            print(f"   Median: {valid_momentum.median():.4f}")
            print(f"   Std Dev: {valid_momentum.std():.4f}")
            print(f"   Uptrend: {(df['price_trend'] == 1).sum()} events")
            print(f"   Downtrend: {(df['price_trend'] == 0).sum()} events")
    
    return df

def create_industry_dummies(df: pd.DataFrame) -> pd.DataFrame:
    """Create industry dummy variables based on ticker"""
    print("Adding industry control variables...")
    
    df = df.copy()
    
    # Industry classification based on ticker knowledge
    tech_companies = ['MSFT', 'GOOG', 'FB', 'INTC', 'ADBE', 'CRM', 'INTU', 'TSLA', 'ORCL', 'NFLX']
    financial_companies = ['BAC', 'JPM', 'MS', 'GS', 'AXP', 'BLK', 'V', 'MA', 'MCO']
    healthcare_companies = ['JNJ', 'ABT', 'LLY', 'AMGN', 'BMY', 'MRK', 'TMO', 'ISRG', 'MDT', 'UNH']
    energy_companies = ['XOM']
    consumer_companies = ['MCD', 'NKE', 'COST', 'HD', 'PG', 'PEP']
    
    # Create industry dummies
    df['tech'] = df['ticker'].isin(tech_companies).astype(int)
    df['financial'] = df['ticker'].isin(financial_companies).astype(int)
    df['healthcare'] = df['ticker'].isin(healthcare_companies).astype(int)
    df['energy'] = df['ticker'].isin(energy_companies).astype(int)
    df['consumer'] = df['ticker'].isin(consumer_companies).astype(int)
    df['other'] = (~df['ticker'].isin(
        tech_companies + financial_companies + healthcare_companies + 
        energy_companies + consumer_companies)).astype(int)
    
    print(" Added industry dummies: tech, financial, healthcare, energy, consumer, other")
    return df

def add_abnormal_turnover_control(df: pd.DataFrame, output_dir: str = "exports/tables") -> pd.DataFrame:
    """
    Add abnormal turnover control variables based on trading volume.
    
    Calculates:
    - Normal turnover: Average volume during estimation window (-60 to -10 days)
    - Abnormal turnover: Event window volume vs normal volume
    - Turnover ratio: Event volume / Normal volume
    """
    print("\nAdding abnormal turnover control variables...")
    
    df = df.copy()
    
    # Load market cap data with volume
    market_data_path = os.path.join(output_dir, "Market_Cap_Daily_Data.csv")
    if not os.path.exists(market_data_path):
        print(f"  Warning: Market cap data not found at {market_data_path}")
        return df
    
    market_data = pd.read_csv(market_data_path)
    market_data['date'] = pd.to_datetime(market_data['date'])
    
    # Check if volume column exists
    if 'volume' not in market_data.columns:
        print("  Warning: Volume column not found in market data")
        return df
    
    print(f"   Loaded market data with {len(market_data)} rows")
    
    # Initialize columns
    df['normal_volume'] = np.nan
    df['event_volume'] = np.nan
    df['abnormal_turnover'] = np.nan
    df['turnover_ratio'] = np.nan
    df['high_turnover'] = 0  # Binary: 1 if turnover ratio > median
    
    # Calculate for each company-event
    for idx, row in df.iterrows():
        ticker = row['ticker']
        event_date = pd.to_datetime(row['event_date'])
        
        # Get company's volume data
        company_data = market_data[market_data['ticker'] == ticker].copy()
        company_data = company_data.sort_values('date')
        
        # 1. Calculate normal volume (estimation window: -60 to -10 days)
        estimation_start = event_date - pd.Timedelta(days=60)
        estimation_end = event_date - pd.Timedelta(days=10)
        
        estimation_data = company_data[
            (company_data['date'] >= estimation_start) & 
            (company_data['date'] <= estimation_end)
        ]
        
        if len(estimation_data) > 0:
            normal_vol = estimation_data['volume'].mean()
            df.at[idx, 'normal_volume'] = normal_vol
        else:
            continue
        
        # 2. Calculate event window volume
        # Use CAR(-1,1) window as primary event window
        event_start = event_date - pd.Timedelta(days=1)
        event_end = event_date + pd.Timedelta(days=1)
        
        event_data = company_data[
            (company_data['date'] >= event_start) & 
            (company_data['date'] <= event_end)
        ]
        
        if len(event_data) > 0:
            event_vol = event_data['volume'].mean()
            df.at[idx, 'event_volume'] = event_vol
            
            # 3. Calculate abnormal turnover metrics
            if normal_vol > 0:
                # Turnover ratio
                turnover_ratio = event_vol / normal_vol
                df.at[idx, 'turnover_ratio'] = turnover_ratio
                
                # Abnormal turnover (percentage change)
                abnormal = ((event_vol - normal_vol) / normal_vol) * 100
                df.at[idx, 'abnormal_turnover'] = abnormal
    
    # 4. Create high turnover binary indicator (above median)
    median_turnover = df['turnover_ratio'].median()
    df['high_turnover'] = (df['turnover_ratio'] > median_turnover).astype(int)
    
    # 5. Create interaction terms with sentiment
    if 'sentiment_score' in df.columns:
        df['abnormal_turnover_x_sentiment'] = df['abnormal_turnover'] * df['sentiment_score']
        df['high_turnover_x_sentiment'] = df['high_turnover'] * df['sentiment_score']
    
    # Print summary statistics
    print(f"\n   Abnormal Turnover Summary:")
    print(f"   Normal volume (mean): {df['normal_volume'].mean():,.0f}")
    print(f"   Event volume (mean): {df['event_volume'].mean():,.0f}")
    print(f"   Turnover ratio (mean): {df['turnover_ratio'].mean():.2f}")
    print(f"   Abnormal turnover % (mean): {df['abnormal_turnover'].mean():.2f}%")
    print(f"   High turnover events: {df['high_turnover'].sum()} ({df['high_turnover'].mean()*100:.1f}%)")
    
    # Distribution of abnormal turnover
    print(f"\n   Abnormal Turnover Distribution:")
    print(f"   Min: {df['abnormal_turnover'].min():.2f}%")
    print(f"   25th percentile: {df['abnormal_turnover'].quantile(0.25):.2f}%")
    print(f"   Median: {df['abnormal_turnover'].median():.2f}%")
    print(f"   75th percentile: {df['abnormal_turnover'].quantile(0.75):.2f}%")
    print(f"   Max: {df['abnormal_turnover'].max():.2f}%")
    
    print(" Added abnormal turnover controls: normal_volume, event_volume, abnormal_turnover, turnover_ratio, high_turnover")
    
    return df

def add_all_controls(df: pd.DataFrame = None, output_dir: str = "exports/tables") -> pd.DataFrame:
    """Add all control variables to the dataset"""
    print("=" * 80)
    print("ADDING CONTROL VARIABLES (WITH REAL MARKET CAP DATA)")
    print("=" * 80)
    
    if df is None:
        df = load_merged_data(output_dir)
    
    # Add all control variable categories (REAL market cap first!)
    df = add_real_firm_size_control(df, output_dir)
    df = add_volatility_control(df)
    df = add_temporal_controls(df)
    df = add_performance_controls(df)
    df = add_sentiment_complexity_controls(df)
    df = add_stock_price_momentum_controls(df, output_dir)  # Pre-event price momentum
    df = add_abnormal_turnover_control(df, output_dir)  # NEW: Abnormal trading volume
    df = create_industry_dummies(df)
    
    # Remove any infinite or extremely large values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)
    
    print(f"\n All control variables added with REAL market cap data!")
    print(f" Final dataset: {len(df)} records")
    print(f" Total variables: {len(df.columns)}")
    
    # Show available CAR windows in final dataset
    car_windows = [col for col in df.columns if col.startswith('car_') and col.count('_') == 2]
    if car_windows:
        print(f" CAR windows in final dataset: {', '.join([c.upper() for c in car_windows])}")
    
    return df

def step6_add_control_variables():
    """Main function to add control variables"""
    
    # Add all controls
    df_with_controls = add_all_controls()
    
    # Save final dataset
    output_path = os.path.join("exports/tables", "Final_Regression_Dataset.csv")
    df_with_controls.to_csv(output_path, index=False)
    
    print(f"\n Final dataset saved to: {output_path}")
    print("=" * 80)
    print(" CONTROL VARIABLES MODULE COMPLETED!")
    print("=" * 80)
    
    return df_with_controls

step6_add_control_variables()

ADDING CONTROL VARIABLES (WITH REAL MARKET CAP DATA)
 Using CAR(-1,1) as primary CAR measure
 Loaded merged dataset: 4348 records
 Available CAR windows: CAR_-1_1, CAR_0_1, CAR_0_2
Adding firm size control variable (REAL Market Capitalization from Yahoo Finance)...
 Loaded real market cap data: 12065 records for 59 companies
   Date range: 2004-11-01 00:00:00 to 2022-03-01 00:00:00
Matching market cap data to earnings events...
  Processed 500/4348 records...
  Processed 1000/4348 records...
  Processed 1500/4348 records...
  Processed 2000/4348 records...
  Processed 2500/4348 records...
  Processed 3000/4348 records...
  Processed 3500/4348 records...
  Processed 4000/4348 records...
 Market cap matching completed: 4348 → 4285 records
Creating additional firm size measures...
 Added REAL firm size controls: market_cap, log_market_cap, market_cap_quintile, firm_size_rank, large_firm

Market Cap Summary ($ Billions):
  Mean: $82.3B
  Median: $56.8B
  Min: $0.8B
  Max: $1716.9B
Adding v

,ticker,quarter,fiscal_year,event_date,transcript_id,total_words,positive_count,negative_count,uncertainty_count,positive_ratio,...,event_volume,abnormal_turnover,turnover_ratio,high_turnover,tech,financial,healthcare,energy,consumer,other
0,ABT,2,2007,2007-07-18 19:16:50,ABT_2330,4580,122,82,71,0.026638,...,17396415.0,44.803069,1.448031,0,0,0,1,0,0,0
1,ABT,3,2007,2007-10-17 17:53:08,ABT_3153,4412,126,73,51,0.028558,...,20467694.5,92.405619,1.924056,1,0,0,1,0,0,0
2,ABT,4,2007,2008-01-23 21:45:06,ABT_4880,4882,167,75,74,0.034207,...,22273343.0,73.651755,1.736518,1,0,0,1,0,0,0
3,ABT,1,2008,2008-04-16 18:24:19,ABT_7370,4680,131,76,70,0.027991,...,28611296.5,121.282353,2.212824,1,0,0,1,0,0,0
4,ABT,2,2008,2008-07-17 00:49:23,ABT_10085,5096,184,70,70,0.036107,...,21447061.5,77.631008,1.776310,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4343,XOM,1,2020,2020-05-01 20:00:40,XOM_167742,1616,62,59,22,0.038366,...,35399300.0,-27.640900,0.723591,0,0,0,0,1,0,0
4344,XOM,0,2020,2020-05-27 21:59:15,XOM_170399,5579,168,192,68,0.030113,...,20243150.0,-40.748284,0.592517,0,0,0,0,1,0,0
4345,XOM,2,2020,2020-07-31 17:59:08,XOM_172110,5973,216,200,88,0.036163,...,32116200.0,33.263378,1.332634,0,0,0,0,1,0,0
4346,XOM,3,2020,2020-10-30 21:24:08,XOM_176630,5304,181,156,61,0.034125,...,47885700.0,77.031703,1.770317,1,0,0,0,1,0,0


# Step 7: Regression Analysis

**Input Files:**
- `exports/tables/Final_Regression_Dataset.csv` (from Step 6)

**Output Files:**
- `exports/tables/Complete_Regression_Analysis.xlsx`
- `exports/tables/Regression_Results_Table.csv`
- `exports/figures/Sentiment_CAR_Scatterplots.png`
- `exports/figures/Regression_Diagnostics.png`

**Analysis:**
- OLS regression with HC3 robust standard errors
- Multiple CAR windows: CAR(-1,1), CAR(0,1), CAR(0,2)
- Diagnostic plots and residual analysis

This module performs the final regression analysis to examine how sentiment
from earnings conference calls affects Cumulative Abnormal Returns (CAR).


In [7]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Install required packages if not available
try:
    import statsmodels.api as sm
    from statsmodels.stats.diagnostic import het_breuschpagan
    from statsmodels.stats.outliers_influence import variance_inflation_factor
except ImportError:
    print("Installing required statistics packages...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'statsmodels'])
    import statsmodels.api as sm
    from statsmodels.stats.diagnostic import het_breuschpagan
    from statsmodels.stats.outliers_influence import variance_inflation_factor

def load_final_dataset(output_dir: str = "exports/tables") -> pd.DataFrame:
    """Load the final dataset with all controls"""
    data_path = os.path.join(output_dir, "Final_Regression_Dataset.csv")
    
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Final dataset not found: {data_path}")
    
    df = pd.read_csv(data_path)
    df['event_date'] = pd.to_datetime(df['event_date'])
    
    print(f" Loaded final dataset: {len(df)} observations")
    return df

def prepare_regression_data(df: pd.DataFrame) -> tuple:
    """Prepare data for regression analysis"""
    
    # Remove rows with missing values in key variables
    key_vars = ['car', 'negative_ratio', 'positive_ratio', 
               'uncertainty_ratio', 'volatility_combined', 'log_market_cap']
    
    df_clean = df.dropna(subset=key_vars).copy()
    
    print(f" Clean dataset: {len(df_clean)} observations ({len(df_clean)/len(df)*100:.1f}%)")
    
    return df_clean

def run_baseline_regression(df: pd.DataFrame) -> dict:
    """Run baseline regression with sentiment variables only"""
    
    print("\n" + "=" * 60)
    print("BASELINE REGRESSION: SENTIMENT ONLY")
    print("=" * 60)
    
    # Define variables
    y = df['car']
    X = df[['negative_ratio', 'positive_ratio', 'uncertainty_ratio']]
    X = sm.add_constant(X)
    
    # Run regression
    model = sm.OLS(y, X).fit(cov_type='HC3')  # Robust standard errors
    
    # Store results
    results = {
        'model': model,
        'r_squared': model.rsquared,
        'adj_r_squared': model.rsquared_adj,
        'f_stat': model.fvalue,
        'f_pvalue': model.f_pvalue,
        'n_obs': model.nobs
    }
    
    print(f"R-squared: {results['r_squared']:.4f}")
    print(f"Adjusted R-squared: {results['adj_r_squared']:.4f}")
    print(f"F-statistic: {results['f_stat']:.4f} (p-value: {results['f_pvalue']:.4f})")
    print(f"Observations: {int(results['n_obs'])}")
    
    return results

def run_full_regression(df: pd.DataFrame) -> dict:
    """Run full regression with sentiment and control variables"""
    
    print("\n" + "=" * 60)
    print("FULL REGRESSION: SENTIMENT + CONTROLS")
    print("=" * 60)
    
    # Define variables
    y = df['car']
    
    # Sentiment variables (main variables of interest)
    sentiment_vars = ['negative_ratio', 'positive_ratio', 'uncertainty_ratio']
    
    # Control variables
    control_vars = ['volatility_combined', 'log_market_cap', 'financial_crisis', 'covid_period']
    
    # Add past performance if available
    if 'past_car' in df.columns and not df['past_car'].isna().all():
        control_vars.append('past_car')
    
    # Industry dummies
    industry_vars = [col for col in df.columns if col.startswith('industry_') and col != 'industry_other']
    
    # Combine all variables
    X_vars = sentiment_vars + control_vars + industry_vars
    X = df[X_vars].copy()
    
    # Handle missing values
    X = X.fillna(X.mean())
    
    # Add constant
    X = sm.add_constant(X)
    
    # Run regression
    model = sm.OLS(y, X).fit(cov_type='HC3')  # Robust standard errors
    
    # Store results
    results = {
        'model': model,
        'r_squared': model.rsquared,
        'adj_r_squared': model.rsquared_adj,
        'f_stat': model.fvalue,
        'f_pvalue': model.f_pvalue,
        'n_obs': model.nobs,
        'sentiment_vars': sentiment_vars,
        'control_vars': control_vars,
        'industry_vars': industry_vars
    }
    
    print(f"R-squared: {results['r_squared']:.4f}")
    print(f"Adjusted R-squared: {results['adj_r_squared']:.4f}")
    print(f"F-statistic: {results['f_stat']:.4f} (p-value: {results['f_pvalue']:.4f})")
    print(f"Observations: {int(results['n_obs'])}")
    
    return results

def run_robustness_tests(df: pd.DataFrame) -> dict:
    """Run robustness tests with different specifications"""
    
    print("\n" + "=" * 60)
    print("ROBUSTNESS TESTS")
    print("=" * 60)
    
    robustness_results = {}
    y = df['car']
    
    # Test 1: Net sentiment measure only
    X1 = df[['net_sentiment', 'uncertainty_ratio', 'volatility_combined', 'log_market_cap']]
    X1 = X1.fillna(X1.mean())
    X1 = sm.add_constant(X1)
    model1 = sm.OLS(y, X1).fit(cov_type='HC3')
    robustness_results['net_sentiment_only'] = model1
    
    # Test 2: Sentiment polarity measure only  
    X2 = df[['sentiment_polarity', 'uncertainty_ratio', 'volatility_combined', 'log_market_cap']]
    X2 = X2.fillna(X2.mean())
    X2 = sm.add_constant(X2)
    model2 = sm.OLS(y, X2).fit(cov_type='HC3')
    robustness_results['sentiment_polarity_only'] = model2
    
    # Test 3: Binary sentiment measure
    df_temp = df.copy()
    df_temp['high_negative'] = (df_temp['negative_ratio'] > df_temp['negative_ratio'].median()).astype(int)
    df_temp['high_positive'] = (df_temp['positive_ratio'] > df_temp['positive_ratio'].median()).astype(int)
    X3 = df_temp[['high_negative', 'high_positive', 'uncertainty_ratio', 
            'volatility_combined', 'log_market_cap']]
    X3 = X3.fillna(X3.mean())
    X3 = sm.add_constant(X3)
    model3 = sm.OLS(y, X3).fit(cov_type='HC3')
    robustness_results['polarity'] = model3
    
    print(" Completed robustness tests:")
    print(f"  - Net sentiment R²: {model1.rsquared:.4f}")
    print(f"  - Sentiment polarity R²: {model2.rsquared:.4f}")
    print(f"  - Binary sentiment R²: {model3.rsquared:.4f}")
    
    return robustness_results

def create_regression_table(baseline_results: dict, full_results: dict, 
                           robustness_results: dict, output_dir: str = "exports/tables") -> pd.DataFrame:
    """Create a comprehensive regression results table"""
    
    print("\nCreating regression results table...")
    
    # Extract coefficients and statistics
    models = {
        'Baseline': baseline_results['model'],
        'Full Model': full_results['model'],
        'Net Sentiment': robustness_results['net_sentiment_only'],
        'Sentiment Polarity': robustness_results['sentiment_polarity_only'],
            'Binary Sentiment': robustness_results['polarity']
    }
    
    # Create results table
    results_data = []
    
    for model_name, model in models.items():
        model_data = {
            'Model': model_name,
            'R²': f"{model.rsquared:.4f}",
            'Adj. R²': f"{model.rsquared_adj:.4f}",
            'F-stat': f"{model.fvalue:.2f}",
            'N': int(model.nobs)
        }
        
        # Add key coefficients
        params = model.params
        pvalues = model.pvalues
        
        for var in ['negative_ratio', 'positive_ratio', 'uncertainty_ratio']:
            if var in params.index:
                coef = params[var]
                pval = pvalues[var]
                significance = '***' if pval < 0.01 else '**' if pval < 0.05 else '*' if pval < 0.1 else ''
                model_data[var] = f"{coef:.4f}{significance}"
            else:
                model_data[var] = '-'
        
        results_data.append(model_data)
    
    results_df = pd.DataFrame(results_data)
    
    # Save results table
    table_path = os.path.join(output_dir, "Regression_Results_Table.csv")
    results_df.to_csv(table_path, index=False)
    
    print(f" Regression table saved: {table_path}")
    return results_df

def create_visualizations(df: pd.DataFrame, baseline_results: dict, full_results: dict, figure_dir: str = "exports/figures"):
    """Create regression visualization plots"""
    
    print("\nCreating visualization plots...")
    
    # Set up plotting style
    plt.style.use('default')
    sns.set_palette("husl")
    
    # Figure 1: Sentiment vs CAR scatter plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Sentiment Measures vs Cumulative Abnormal Returns', fontsize=16, fontweight='bold')
    
    # Negative sentiment
    sns.scatterplot(data=df, x='negative_ratio', y='car', alpha=0.6, ax=axes[0,0])
    sns.regplot(data=df, x='negative_ratio', y='car', scatter=False, color='red', ax=axes[0,0])
    axes[0,0].set_title('Negative Sentiment vs CAR')
    axes[0,0].set_xlabel('Negative Sentiment Ratio')
    axes[0,0].set_ylabel('CAR (-1,+1)')
    
    # Positive sentiment  
    sns.scatterplot(data=df, x='positive_ratio', y='car', alpha=0.6, ax=axes[0,1])
    sns.regplot(data=df, x='positive_ratio', y='car', scatter=False, color='green', ax=axes[0,1])
    axes[0,1].set_title('Positive Sentiment vs CAR')
    axes[0,1].set_xlabel('Positive Sentiment Ratio')
    axes[0,1].set_ylabel('CAR (-1,+1)')
    
    # Uncertainty
    sns.scatterplot(data=df, x='uncertainty_ratio', y='car', alpha=0.6, ax=axes[1,0])
    sns.regplot(data=df, x='uncertainty_ratio', y='car', scatter=False, color='orange', ax=axes[1,0])
    axes[1,0].set_title('Uncertainty vs CAR')
    axes[1,0].set_xlabel('Uncertainty Ratio')
    axes[1,0].set_ylabel('CAR (-1,+1)')
    
    # Sentiment polarity
    sns.scatterplot(data=df, x='sentiment_polarity', y='car', alpha=0.6, ax=axes[1,1])
    sns.regplot(data=df, x='sentiment_polarity', y='car', scatter=False, color='purple', ax=axes[1,1])
    axes[1,1].set_title('Sentiment Polarity vs CAR')
    axes[1,1].set_xlabel('Sentiment Polarity')
    axes[1,1].set_ylabel('CAR (-1,+1)')
    
    plt.tight_layout()
    os.makedirs(figure_dir, exist_ok=True)
    scatter_path = os.path.join(figure_dir, "Sentiment_CAR_Scatterplots.png")
    plt.savefig(scatter_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Figure 2: Residual plots
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Regression Diagnostics', fontsize=16, fontweight='bold')
    
    # Residuals vs fitted
    fitted = full_results['model'].fittedvalues
    residuals = full_results['model'].resid
    
    axes[0].scatter(fitted, residuals, alpha=0.6)
    axes[0].axhline(y=0, color='red', linestyle='--')
    axes[0].set_xlabel('Fitted Values')
    axes[0].set_ylabel('Residuals')
    axes[0].set_title('Residuals vs Fitted Values')
    
    # Q-Q plot
    stats.probplot(residuals, dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot of Residuals')
    
    plt.tight_layout()
    diagnostics_path = os.path.join(figure_dir, "Regression_Diagnostics.png")
    plt.savefig(diagnostics_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f" Plots saved:")
    print(f"  - {scatter_path}")
    print(f"  - {diagnostics_path}")

def save_detailed_results(baseline_results: dict, full_results: dict, 
                         robustness_results: dict, results_table: pd.DataFrame, output_dir: str = "exports/tables"):
    """Save detailed regression results to Excel"""
    
    excel_path = os.path.join(output_dir, "Complete_Regression_Analysis.xlsx")
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Results summary table
        results_table.to_excel(writer, sheet_name='Results_Summary', index=False)
        
        # Detailed coefficients for main model
        full_model = full_results['model']
        coef_table = pd.DataFrame({
            'Variable': full_model.params.index,
            'Coefficient': full_model.params.values,
            'Std_Error': full_model.bse.values,
            't_statistic': full_model.tvalues.values,
            'p_value': full_model.pvalues.values,
            'CI_lower': full_model.conf_int()[0].values,
            'CI_upper': full_model.conf_int()[1].values
        })
        coef_table.to_excel(writer, sheet_name='Detailed_Coefficients', index=False)
        
        # Model comparison
        comparison_data = []
        models = {'Baseline': baseline_results, 'Full': full_results}
        
        for name, results in models.items():
            model = results['model']
            comparison_data.append({
                'Model': name,
                'R_squared': model.rsquared,
                'Adj_R_squared': model.rsquared_adj,
                'F_statistic': model.fvalue,
                'F_p_value': model.f_pvalue,
                'AIC': model.aic,
                'BIC': model.bic,
                'Log_Likelihood': model.llf,
                'Observations': int(model.nobs)
            })
        
        comparison_df = pd.DataFrame(comparison_data)
        comparison_df.to_excel(writer, sheet_name='Model_Comparison', index=False)
    
    print(f" Detailed results saved: {excel_path}")

def print_interpretation(full_results: dict):
    """Print interpretation of results"""
    
    print("\n" + "=" * 80)
    print("REGRESSION RESULTS INTERPRETATION")
    print("=" * 80)
    
    model = full_results['model']
    params = model.params
    pvalues = model.pvalues
    
    print(f"Model explains {model.rsquared*100:.2f}% of CAR variation")
    print(f"F-test p-value: {model.f_pvalue:.4f} {'(Significant)' if model.f_pvalue < 0.05 else '(Not significant)'}")
    
    print(f"\nKey Findings:")
    
    # Negative sentiment
    if 'negative_ratio' in params.index:
        coef = params['negative_ratio']
        pval = pvalues['negative_ratio']
        sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
        direction = "increases" if coef > 0 else "decreases"
        print(f"• Negative sentiment {direction} CAR by {abs(coef):.4f} {sig}")
        if sig:
            print(f"  → 1 percentage point increase in negative sentiment leads to {coef*100:.2f} percentage point change in CAR")
    
    # Positive sentiment
    if 'positive_ratio' in params.index:
        coef = params['positive_ratio']
        pval = pvalues['positive_ratio'] 
        sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
        direction = "increases" if coef > 0 else "decreases"
        print(f"• Positive sentiment {direction} CAR by {abs(coef):.4f} {sig}")
        if sig:
            print(f"  → 1 percentage point increase in positive sentiment leads to {coef*100:.2f} percentage point change in CAR")
    
    # Uncertainty
    if 'uncertainty_ratio' in params.index:
        coef = params['uncertainty_ratio']
        pval = pvalues['uncertainty_ratio']
        direction = "increases" if coef > 0 else "decreases" 
        print(f"• Uncertainty {direction} CAR by {abs(coef):.4f} {sig}")
        if sig:
            print(f"  → 1 percentage point increase in uncertainty leads to {coef*100:.2f} percentage point change in CAR")
    
    print(f"\nStatistical significance: *** p<0.01, ** p<0.05, * p<0.1")


def run_regressions_for_all_car_windows(df: pd.DataFrame, output_dir: str = "exports/tables") -> dict:
    """
    Run regression analysis for all available CAR windows
    Returns results for each CAR window
    """
    print("\n" + "=" * 80)
    print("RUNNING REGRESSIONS FOR ALL CAR WINDOWS")
    print("=" * 80)
    
    # Identify all available CAR windows
    car_windows = [col for col in df.columns if col.startswith('car_') and col.count('_') == 2]
    
    if not car_windows:
        print("  No CAR windows found in dataset!")
        return {}
    
    print(f" Found {len(car_windows)} CAR windows: {', '.join([c.upper() for c in car_windows])}\n")
    
    all_results = {}
    
    for car_col in car_windows:
        print(f"\n{'='*80}")
        print(f"ANALYZING {car_col.upper()}")
        print(f"{'='*80}")
        
        # Create temporary df with this CAR as the primary measure
        df_temp = df.copy()
        df_temp['car'] = df_temp[car_col]
        
        # Get corresponding t-stat, alpha, beta, r_squared
        car_tstat_col = f"{car_col}_tstat"
        car_alpha_col = f"{car_col}_alpha"
        car_beta_col = f"{car_col}_beta"
        car_rsquared_col = f"{car_col}_rsquared"
        
        if car_tstat_col in df_temp.columns:
            df_temp['car_tstat'] = df_temp[car_tstat_col]
        if car_alpha_col in df_temp.columns:
            df_temp['alpha'] = df_temp[car_alpha_col]
        if car_beta_col in df_temp.columns:
            df_temp['beta'] = df_temp[car_beta_col]
        if car_rsquared_col in df_temp.columns:
            df_temp['r_squared'] = df_temp[car_rsquared_col]
        
        # Prepare data
        df_clean = prepare_regression_data(df_temp)
        
        # Run regressions
        try:
            baseline_results = run_baseline_regression(df_clean)
            full_results = run_full_regression(df_clean)
            robustness_results = run_robustness_tests(df_clean)
            
            all_results[car_col] = {
                'baseline': baseline_results,
                'full': full_results,
                'robustness': robustness_results,
                'car_window': car_col
            }
            
            # Print key results
            model = full_results['model']
            print(f"\n {car_col.upper()} Results:")
            print(f"   R²: {model.rsquared:.4f}")
            print(f"   Adj. R²: {model.rsquared_adj:.4f}")
            print(f"   F-statistic: {model.fvalue:.4f} (p={model.f_pvalue:.4f})")
            print(f"   N: {int(model.nobs)}")
            
            # Key coefficients
            if 'positive_ratio' in model.params:
                coef = model.params['positive_ratio']
                pval = model.pvalues['positive_ratio']
                sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
                print(f"   Positive sentiment: {coef:.4f} {sig}")
            
            if 'negative_ratio' in model.params:
                coef = model.params['negative_ratio']
                pval = model.pvalues['negative_ratio']
                sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
                print(f"   Negative sentiment: {coef:.4f} {sig}")
                
        except Exception as e:
            print(f" Error analyzing {car_col}: {str(e)}")
            continue
    
    # Save comparative results
    if all_results:
        save_multi_car_results(all_results, output_dir)
    
    return all_results


def save_multi_car_results(all_results: dict, output_dir: str = "exports/tables"):
    """Save comparative results for all CAR windows"""
    print(f"\n{'='*80}")
    print("SAVING MULTI-CAR WINDOW RESULTS")
    print(f"{'='*80}")
    
    # Create comparison table
    comparison_data = []
    
    for car_window, results in all_results.items():
        model = results['full']['model']
        
        row = {
            'CAR_Window': car_window.upper(),
            'R_Squared': model.rsquared,
            'Adj_R_Squared': model.rsquared_adj,
            'F_Statistic': model.fvalue,
            'F_PValue': model.f_pvalue,
            'N_Obs': int(model.nobs)
        }
        
        # Add coefficients
        for var in ['positive_ratio', 'negative_ratio', 'uncertainty_ratio']:
            if var in model.params:
                row[f'{var}_coef'] = model.params[var]
                row[f'{var}_pval'] = model.pvalues[var]
                row[f'{var}_sig'] = "***" if model.pvalues[var] < 0.01 else "**" if model.pvalues[var] < 0.05 else "*" if model.pvalues[var] < 0.1 else ""
        
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Save to Excel
    excel_path = os.path.join(output_dir, "Multi_CAR_Window_Comparison.xlsx")
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        comparison_df.to_excel(writer, sheet_name='Comparison', index=False)
        
        # Add detailed results for each window
        for car_window, results in all_results.items():
            sheet_name = car_window.replace('car_', 'CAR_').replace('_', ',')[:31]  # Excel limit
            model_summary = results['full']['model'].summary2().tables[1]
            model_summary.to_excel(writer, sheet_name=sheet_name)
    
    print(f" Saved multi-CAR comparison to: {excel_path}")
    
    # Save CSV
    csv_path = os.path.join(output_dir, "Multi_CAR_Window_Comparison.csv")
    comparison_df.to_csv(csv_path, index=False)
    print(f" Saved comparison table to: {csv_path}")


def step7_regression_analysis():
    """Main regression analysis function - supports multiple CAR windows"""
    
    print("=" * 80)
    print("REGRESSION ANALYSIS: SENTIMENT IMPACT ON CAR")
    print("=" * 80)
    
    try:
        # Load data
        df = load_final_dataset()
        
        # Check for multiple CAR windows
        car_windows = [col for col in df.columns if col.startswith('car_') and col.count('_') == 2]
        
        if len(car_windows) > 1:
            print(f"\n Multiple CAR windows detected: {', '.join([c.upper() for c in car_windows])}")
            print("Running analysis for ALL CAR windows...\n")
            
            # Run regressions for all CAR windows
            all_car_results = run_regressions_for_all_car_windows(df)
            
            # Also run for primary CAR (backward compatibility)
            print(f"\n{'='*80}")
            print("PRIMARY CAR ANALYSIS (CAR column)")
            print(f"{'='*80}")
            
        df_clean = prepare_regression_data(df)
        
        # Run regressions on primary CAR
        baseline_results = run_baseline_regression(df_clean)
        full_results = run_full_regression(df_clean)
        robustness_results = run_robustness_tests(df_clean)
        
        # Create results table
        results_table = create_regression_table(baseline_results, full_results, robustness_results)
        
        # Create visualizations
        create_visualizations(df_clean, baseline_results, full_results)
        
        # Save detailed results
        save_detailed_results(baseline_results, full_results, robustness_results, results_table)
        
        # Print interpretation
        print_interpretation(full_results)
        
        print("\n" + "=" * 80)
        print(" REGRESSION ANALYSIS COMPLETED!")
        print("=" * 80)
        print("Files created:")
        print("• Complete_Regression_Analysis.xlsx")
        print("• Regression_Results_Table.csv")
        print("• Sentiment_CAR_Scatterplots.png")
        print("• Regression_Diagnostics.png")
        
        if len(car_windows) > 1:
            print("• Multi_CAR_Window_Comparison.xlsx")
            print("• Multi_CAR_Window_Comparison.csv")
        
        print(f"\n Ready for report writing!")
        
    except Exception as e:
        print(f" Error in regression analysis: {str(e)}")
        print("Please ensure the final dataset with controls exists.")
        import traceback
        traceback.print_exc()

step7_regression_analysis()

REGRESSION ANALYSIS: SENTIMENT IMPACT ON CAR
 Loaded final dataset: 4285 observations

 Multiple CAR windows detected: CAR_-1_1, CAR_0_1, CAR_0_2
Running analysis for ALL CAR windows...


RUNNING REGRESSIONS FOR ALL CAR WINDOWS
 Found 3 CAR windows: CAR_-1_1, CAR_0_1, CAR_0_2


ANALYZING CAR_-1_1
 Clean dataset: 4121 observations (96.2%)

BASELINE REGRESSION: SENTIMENT ONLY
R-squared: 0.0028
Adjusted R-squared: 0.0021
F-statistic: 4.3700 (p-value: 0.0044)
Observations: 4121

FULL REGRESSION: SENTIMENT + CONTROLS
R-squared: 0.0086
Adjusted R-squared: 0.0067
F-statistic: 3.3426 (p-value: 0.0008)
Observations: 4121

ROBUSTNESS TESTS
 Completed robustness tests:
  - Net sentiment R²: 0.0064
  - Sentiment polarity R²: 0.0065
  - Binary sentiment R²: 0.0071

 CAR_-1_1 Results:
   R²: 0.0086
   Adj. R²: 0.0067
   F-statistic: 3.3426 (p=0.0008)
   N: 4121
   Positive sentiment: 0.1664 *
   Negative sentiment: -0.3113 ***

ANALYZING CAR_0_1
 Clean dataset: 3319 observations (77.5%)

BASELINE RE

# Step 8: Dashboard Visualization

**Input Files:**
- `exports/tables/Final_Regression_Dataset.csv` (from Step 6)
- `exports/tables/Complete_Regression_Analysis.xlsx` (from Step 7)

**Output Files:**
- `exports/figures/Analysis_Dashboard.png` (comprehensive 6-panel dashboard)

**Dashboard Panels:**
1. CAR by sentiment score (all 3 CAR windows)
2. Sentiment score distribution
3. Sentiment polarity vs CAR
4. CAR distribution by industry
5. Market cap vs CAR
6. Residual diagnostics
This script analyzes sentiment-CAR relationships for each company individually
and creates visualizations to show company-specific patterns.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def load_company_data(data_dir: str = "exports/tables") -> pd.DataFrame:
    """Load the final regression dataset"""
    data_path = os.path.join(data_dir, "Final_Regression_Dataset.csv")
    
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Dataset not found: {data_path}")
    
    df = pd.read_csv(data_path)
    df = df.dropna(subset=['car', 'positive_ratio', 'negative_ratio'])
    
    print(f" Loaded data: {len(df)} observations for {df['ticker'].nunique()} companies")
    return df

def analyze_company_correlations(df: pd.DataFrame) -> pd.DataFrame:
    """Calculate correlations for each company"""
    
    results = []
    
    for ticker in sorted(df['ticker'].unique()):
        company_data = df[df['ticker'] == ticker].copy()
        
        if len(company_data) < 5:  # Need minimum observations
            continue
        
        # Calculate correlations
        pos_corr = company_data['positive_ratio'].corr(company_data['car'])
        neg_corr = company_data['negative_ratio'].corr(company_data['car'])
        unc_corr = company_data['uncertainty_ratio'].corr(company_data['car'])
        
        # Calculate regression coefficients
        try:
            from scipy import stats
            pos_slope, pos_intercept, pos_r_value, pos_p_value, pos_std_err = stats.linregress(
                company_data['positive_ratio'], company_data['car'])
            
            neg_slope, neg_intercept, neg_r_value, neg_p_value, neg_std_err = stats.linregress(
                company_data['negative_ratio'], company_data['car'])
            
        except:
            pos_slope = pos_p_value = neg_slope = neg_p_value = np.nan
        
        # Get company info
        avg_market_cap = company_data['market_cap_billions'].mean()
        industry = get_company_industry(ticker)
        
        results.append({
            'ticker': ticker,
            'n_observations': len(company_data),
            'industry': industry,
            'avg_market_cap_b': avg_market_cap,
            'pos_correlation': pos_corr,
            'neg_correlation': neg_corr,
            'unc_correlation': unc_corr,
            'pos_slope': pos_slope,
            'neg_slope': neg_slope,
            'pos_p_value': pos_p_value,
            'neg_p_value': neg_p_value,
            'mean_car': company_data['car'].mean(),
            'std_car': company_data['car'].std(),
            'mean_pos_sentiment': company_data['positive_ratio'].mean(),
            'std_pos_sentiment': company_data['positive_ratio'].std()
        })
    
    results_df = pd.DataFrame(results)
    
    # Add significance indicators
    results_df['pos_significant'] = results_df['pos_p_value'] < 0.05
    results_df['neg_significant'] = results_df['neg_p_value'] < 0.05
    
    print(f" Analyzed {len(results_df)} companies with sufficient data")
    
    return results_df

def get_company_industry(ticker: str) -> str:
    """Classify company by industry"""
    industries = {
        'Technology': ['MSFT', 'GOOG', 'FB', 'INTC', 'ADBE', 'CRM', 'INTU', 'TSLA', 'ORCL', 'NFLX'],
        'Financial': ['BAC', 'JPM', 'MS', 'GS', 'AXP', 'BLK', 'V', 'MA', 'MCO'],
        'Healthcare': ['JNJ', 'ABT', 'LLY', 'AMGN', 'BMY', 'MRK', 'TMO', 'ISRG', 'MDT', 'UNH'],
        'Energy': ['XOM'],
        'Consumer': ['MCD', 'NKE', 'COST', 'HD', 'PG', 'PEP'],
        'Industrial': ['BA', 'HON', 'UNP', 'LMT', 'FDX', 'UPS'],
        'Telecom': ['T', 'VZ', 'TMUS']
    }
    
    for industry, tickers in industries.items():
        if ticker in tickers:
            return industry
    return 'Other'

def create_company_grid_plot(df: pd.DataFrame, results_df: pd.DataFrame, figure_dir: str = "exports/figures"):
        """Create SEPARATE figures for each company with 3 plots: positive, negative, uncertainty vs CAR"""
        
        # Select top companies by positive correlation and significance
        top_pos_companies = results_df[
            (results_df['n_observations'] >= 10) & 
            (results_df['pos_significant'] == True)
        ].nlargest(6, 'pos_correlation')
        
        # Select top companies by negative correlation and significance  
        top_neg_companies = results_df[
            (results_df['n_observations'] >= 10) & 
            (results_df['neg_significant'] == True)
        ].nsmallest(6, 'neg_correlation')
        
        # If not enough significant companies, use top correlations regardless
        if len(top_pos_companies) < 3:
            print(" Not enough companies with significant positive relationships")
            top_pos_companies = results_df[results_df['n_observations'] >= 10].nlargest(6, 'pos_correlation')
            
        if len(top_neg_companies) < 3:
            print(" Not enough companies with significant negative relationships")
            top_neg_companies = results_df[results_df['n_observations'] >= 10].nsmallest(6, 'neg_correlation')
        
        # Combine for display
        top_companies = pd.concat([top_pos_companies, top_neg_companies]).drop_duplicates()
        
        # Create separate directory for individual company plots
        company_plots_dir = os.path.join(figure_dir, "company_specific")
        os.makedirs(company_plots_dir, exist_ok=True)
        
        saved_files = []
        
        print(f" Creating individual company figures (3 plots each: Positive, Negative, Uncertainty)...")
        
        # Create SEPARATE figure for EACH company with 3 subplots
        for i, (_, company_info) in enumerate(top_companies.iterrows()):
            ticker = company_info['ticker']
            company_data = df[df['ticker'] == ticker].copy()
            
            # Create figure with 3 subplots (1 row, 3 columns)
            fig, axes = plt.subplots(1, 3, figsize=(18, 5))
            
            # ===== PLOT 1: POSITIVE SENTIMENT =====
            ax_pos = axes[0]
            ax_pos.scatter(company_data['positive_ratio'], 
                          company_data['car'], 
                          alpha=0.6, s=50, color='#2E7D32', edgecolors='white', linewidth=0.5, zorder=2)
            
            # Positive sentiment trend line
            if len(company_data) > 1:
                valid_mask = company_data['positive_ratio'].notna() & company_data['car'].notna()
                if valid_mask.sum() > 1:
                    z = np.polyfit(company_data.loc[valid_mask, 'positive_ratio'], 
                                  company_data.loc[valid_mask, 'car'], 1)
                    p = np.poly1d(z)
                    x_line = np.linspace(company_data['positive_ratio'].min(), 
                                       company_data['positive_ratio'].max(), 100)
                    ax_pos.plot(x_line, p(x_line), color='black', linestyle='--', 
                               linewidth=3, dashes=(5, 3), alpha=0.8, label='Trend Line', zorder=3)
            
            # Positive formatting
            pos_correlation = company_info['pos_correlation']
            pos_p_value = company_info['pos_p_value']
            pos_significance = "***" if pos_p_value < 0.001 else "**" if pos_p_value < 0.01 else "*" if pos_p_value < 0.05 else ""
            
            ax_pos.set_title(f'POSITIVE Sentiment\nr={pos_correlation:.3f}{pos_significance}', 
                            fontsize=12, fontweight='bold', color='#2E7D32')
            ax_pos.set_xlabel('Positive Sentiment Ratio', fontsize=11, fontweight='bold')
            ax_pos.set_ylabel('CAR (-1,+1)', fontsize=11, fontweight='bold')
            ax_pos.grid(True, alpha=0.3, zorder=1)
            ax_pos.axhline(y=0, color='gray', linestyle='-', linewidth=0.8, alpha=0.5)
            if len(company_data) > 1:
                ax_pos.legend(loc='best', framealpha=0.9)
            
            # ===== PLOT 2: NEGATIVE SENTIMENT =====
            ax_neg = axes[1]
            ax_neg.scatter(company_data['negative_ratio'], 
                          company_data['car'], 
                          alpha=0.6, s=50, color='#C62828', edgecolors='white', linewidth=0.5, zorder=2)
            
            # Negative sentiment trend line
            if len(company_data) > 1:
                valid_mask = company_data['negative_ratio'].notna() & company_data['car'].notna()
                if valid_mask.sum() > 1:
                    z = np.polyfit(company_data.loc[valid_mask, 'negative_ratio'], 
                                  company_data.loc[valid_mask, 'car'], 1)
                    p = np.poly1d(z)
                    x_line = np.linspace(company_data['negative_ratio'].min(), 
                                       company_data['negative_ratio'].max(), 100)
                    ax_neg.plot(x_line, p(x_line), color='black', linestyle='--', 
                               linewidth=3, dashes=(5, 3), alpha=0.8, label='Trend Line', zorder=3)
            
            # Negative formatting
            neg_correlation = company_info['neg_correlation']
            neg_p_value = company_info['neg_p_value']
            neg_significance = "***" if neg_p_value < 0.001 else "**" if neg_p_value < 0.01 else "*" if neg_p_value < 0.05 else ""
            
            ax_neg.set_title(f'NEGATIVE Sentiment\nr={neg_correlation:.3f}{neg_significance}', 
                            fontsize=12, fontweight='bold', color='#C62828')
            ax_neg.set_xlabel('Negative Sentiment Ratio', fontsize=11, fontweight='bold')
            ax_neg.set_ylabel('CAR (-1,+1)', fontsize=11, fontweight='bold')
            ax_neg.grid(True, alpha=0.3, zorder=1)
            ax_neg.axhline(y=0, color='gray', linestyle='-', linewidth=0.8, alpha=0.5)
            if len(company_data) > 1:
                ax_neg.legend(loc='best', framealpha=0.9)
            
            # ===== PLOT 3: UNCERTAINTY =====
            ax_unc = axes[2]
            ax_unc.scatter(company_data['uncertainty_ratio'], 
                          company_data['car'], 
                          alpha=0.6, s=50, color='#F57C00', edgecolors='white', linewidth=0.5, zorder=2)
            
            # Uncertainty trend line
            if len(company_data) > 1:
                valid_mask = company_data['uncertainty_ratio'].notna() & company_data['car'].notna()
                if valid_mask.sum() > 1:
                    z = np.polyfit(company_data.loc[valid_mask, 'uncertainty_ratio'], 
                                  company_data.loc[valid_mask, 'car'], 1)
                    p = np.poly1d(z)
                    x_line = np.linspace(company_data['uncertainty_ratio'].min(), 
                                       company_data['uncertainty_ratio'].max(), 100)
                    ax_unc.plot(x_line, p(x_line), color='black', linestyle='--', 
                               linewidth=3, dashes=(5, 3), alpha=0.8, label='Trend Line', zorder=3)
            
            # Uncertainty correlation
            unc_correlation = company_data[['uncertainty_ratio', 'car']].corr().iloc[0, 1]
            from scipy.stats import pearsonr
            valid_data = company_data[['uncertainty_ratio', 'car']].dropna()
            if len(valid_data) > 2:
                _, unc_p_value = pearsonr(valid_data['uncertainty_ratio'], valid_data['car'])
                unc_significance = "***" if unc_p_value < 0.001 else "**" if unc_p_value < 0.01 else "*" if unc_p_value < 0.05 else ""
            else:
                unc_significance = ""
            
            ax_unc.set_title(f'UNCERTAINTY\nr={unc_correlation:.3f}{unc_significance}', 
                            fontsize=12, fontweight='bold', color='#F57C00')
            ax_unc.set_xlabel('Uncertainty Ratio', fontsize=11, fontweight='bold')
            ax_unc.set_ylabel('CAR (-1,+1)', fontsize=11, fontweight='bold')
            ax_unc.grid(True, alpha=0.3, zorder=1)
            ax_unc.axhline(y=0, color='gray', linestyle='-', linewidth=0.8, alpha=0.5)
            if len(company_data) > 1:
                ax_unc.legend(loc='best', framealpha=0.9)
            
            # Overall title
            industry = company_info['industry']
            n_obs = company_info['n_observations']
            plt.suptitle(f'{ticker} - Sentiment vs CAR Analysis ({industry}, n={n_obs})', 
                        fontsize=16, fontweight='bold', y=1.02)
            
            plt.tight_layout()
            
            # Save individual company plot
            company_plot_path = os.path.join(company_plots_dir, f"{ticker}_Sentiment_Analysis.png")
            plt.savefig(company_plot_path, dpi=300, bbox_inches='tight')
            plt.close()
            
            saved_files.append(company_plot_path)
            print(f"    Saved: {ticker}_Sentiment_Analysis.png")
        
        print(f"\n  Created {len(saved_files)} individual company plots")
        print(f" Location: {company_plots_dir}")
        
        return saved_files

def create_company_summary_plot(results_df: pd.DataFrame, figure_dir: str = "exports/figures"):
        """Create summary plots of company-level results"""
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Plot 1: Distribution of positive AND negative correlations by industry
        ax1 = axes[0, 0]
        industries = results_df['industry'].unique()
        pos_data = []
        neg_data = []
        industry_labels = []
        
        for industry in industries:
            industry_df = results_df[results_df['industry'] == industry]
            if len(industry_df) > 0:
                pos_data.append(industry_df['pos_correlation'].tolist())
                neg_data.append(industry_df['neg_correlation'].tolist())
                industry_labels.append(f"{industry}\n(n={len(industry_df)})")
        
        # Create side-by-side box plots
        positions_pos = np.arange(1, len(industries)*2, 2)
        positions_neg = np.arange(2, len(industries)*2+1, 2)
        
        bp1_pos = ax1.boxplot(pos_data, positions=positions_pos, widths=0.6, 
                             patch_artist=True, boxprops=dict(facecolor='lightgreen'))
        bp1_neg = ax1.boxplot(neg_data, positions=positions_neg, widths=0.6,
                             patch_artist=True, boxprops=dict(facecolor='lightcoral'))
        
        ax1.set_title('Sentiment-CAR Correlations by Industry', fontweight='bold')
        ax1.set_ylabel('Correlation Coefficient')
        ax1.set_xticks(np.arange(1.5, len(industries)*2, 2))
        ax1.set_xticklabels(industry_labels, rotation=45, ha='right')
        ax1.grid(True, alpha=0.3)
        ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        ax1.legend([bp1_pos["boxes"][0], bp1_neg["boxes"][0]], 
                  ['Positive Sentiment', 'Negative Sentiment'], loc='upper right')
        
        # Plot 2: Correlation vs Market Cap
        ax2 = axes[0, 1]
        scatter = ax2.scatter(results_df['avg_market_cap_b'], 
                             results_df['pos_correlation'],
                             c=results_df['n_observations'], 
                             cmap='viridis', 
                             alpha=0.7, s=60)
        ax2.set_xlabel('Average Market Cap ($ Billions)')
        ax2.set_ylabel('Positive Sentiment Correlation')
        ax2.set_title('Sentiment Correlation vs Market Cap', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        plt.colorbar(scatter, ax=ax2, label='# Observations')
        
        # Add company labels for interesting cases
        for _, row in results_df.iterrows():
            if abs(row['pos_correlation']) > 0.3 or row['avg_market_cap_b'] > 500:
                ax2.annotate(row['ticker'], 
                           (row['avg_market_cap_b'], row['pos_correlation']),
                           xytext=(5, 5), textcoords='offset points', 
                           fontsize=8, alpha=0.8)
        
        # Plot 3: Significant vs Non-significant companies (both positive and negative)
        ax3 = axes[1, 0]
        
        # Positive sentiment significance
        sig_pos = results_df[results_df['pos_significant']]['pos_correlation']
        nonsig_pos = results_df[~results_df['pos_significant']]['pos_correlation']
        
        # Negative sentiment significance  
        sig_neg = results_df[results_df['neg_significant']]['neg_correlation']
        nonsig_neg = results_df[~results_df['neg_significant']]['neg_correlation']
        
        # Create histogram
        bins = np.linspace(-0.5, 0.5, 20)
        ax3.hist([sig_pos, nonsig_pos], bins=bins, alpha=0.7, 
                label=[f'Pos Significant (n={len(sig_pos)})', f'Pos Non-sig (n={len(nonsig_pos)})'],
                color=['darkgreen', 'lightgreen'])
        ax3.hist([sig_neg, nonsig_neg], bins=bins, alpha=0.7, 
                label=[f'Neg Significant (n={len(sig_neg)})', f'Neg Non-sig (n={len(nonsig_neg)})'],
                color=['darkred', 'lightcoral'])
        
        ax3.set_xlabel('Sentiment Correlation')
        ax3.set_ylabel('Number of Companies')
        ax3.set_title('Distribution of Sentiment Correlations\n(Significant vs Non-significant)', fontweight='bold')
        ax3.legend(fontsize=8)
        ax3.grid(True, alpha=0.3)
        ax3.axvline(x=0, color='black', linestyle='-', alpha=0.3)
        
        # Plot 4: Top positive and negative sentiment performers
        ax4 = axes[1, 1]
        top_n = 8
        
        # Top positive sentiment performers
        top_pos = results_df.nlargest(top_n, 'pos_correlation')
        # Top negative sentiment performers (most negative correlation)
        top_neg = results_df.nsmallest(top_n, 'neg_correlation')
        
        y_pos_pos = np.arange(len(top_pos))
        y_pos_neg = np.arange(len(top_neg)) - len(top_neg) - 1
        
        # Bars for positive sentiment
        bars_pos = ax4.barh(y_pos_pos, top_pos['pos_correlation'], 
                           color='green', alpha=0.7, label=f'Top {top_n} Positive')
        
        # Bars for negative sentiment  
        bars_neg = ax4.barh(y_pos_neg, top_neg['neg_correlation'], 
                           color='red', alpha=0.7, label=f'Top {top_n} Negative')
        
        # Add significance indicators
        for i, (_, row) in enumerate(top_pos.iterrows()):
            if row['pos_significant']:
                ax4.text(row['pos_correlation'] + 0.01, y_pos_pos[i], '*', 
                        fontsize=16, fontweight='bold', va='center')
        
        for i, (_, row) in enumerate(top_neg.iterrows()):
            if row['neg_significant']:
                ax4.text(row['neg_correlation'] - 0.01, y_pos_neg[i], '*', 
                        fontsize=16, fontweight='bold', va='center')
        
        # Labels
        ax4.set_yticks(list(y_pos_pos) + list(y_pos_neg))
        ax4.set_yticklabels(list(top_pos['ticker']) + list(top_neg['ticker']))
        ax4.set_xlabel('Sentiment Correlation (* = Significant)')
        ax4.set_title(f'Top Companies by Positive & Negative Sentiment Effects', fontweight='bold')
        ax4.axvline(x=0, color='black', linestyle='-', alpha=0.3)
        ax4.grid(True, alpha=0.3)
        ax4.legend()
        
        plt.tight_layout()
        
        # Save plot
        os.makedirs(figure_dir, exist_ok=True)
        summary_path = os.path.join(figure_dir, "Company_Analysis_Summary.png")
        plt.savefig(summary_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f" Company summary plot saved: {summary_path}")
        
        return summary_path

def save_company_results(results_df: pd.DataFrame, data_dir: str = "exports/tables"):
    """Save detailed company-level results"""
    
    # Sort by positive correlation (descending)
    results_df = results_df.sort_values('pos_correlation', ascending=False)
    
    # Save to CSV
    csv_path = os.path.join(data_dir, "Company_Specific_Analysis.csv")
    results_df.to_csv(csv_path, index=False)
    
    # Save to Excel with multiple sheets
    excel_path = os.path.join(data_dir, "Company_Specific_Analysis.xlsx")
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # All results
        results_df.to_excel(writer, sheet_name='All_Companies', index=False)
        
        # Significant positive relationships
        sig_positive = results_df[
            (results_df['pos_significant']) & 
            (results_df['pos_correlation'] > 0)
        ]
        sig_positive.to_excel(writer, sheet_name='Significant_Positive', index=False)
        
        # By industry
        for industry in results_df['industry'].unique():
            industry_data = results_df[results_df['industry'] == industry]
            sheet_name = f'Industry_{industry}'[:31]  # Excel sheet name limit
            industry_data.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f" Company results saved:")
    print(f"  - {csv_path}")
    print(f"  - {excel_path}")
    
    return results_df

def print_key_findings(results_df: pd.DataFrame):
    """Print summary of key findings"""
    
    print("\n" + "=" * 80)
    print("COMPANY-SPECIFIC SENTIMENT ANALYSIS RESULTS")
    print("=" * 80)
    
    total_companies = len(results_df)
    significant_positive = len(results_df[
        (results_df['pos_significant']) & 
        (results_df['pos_correlation'] > 0)
    ])
    significant_negative = len(results_df[
        (results_df['pos_significant']) & 
        (results_df['pos_correlation'] < 0)
    ])
    
    print(f"Total companies analyzed: {total_companies}")
    print(f"Companies with significant POSITIVE sentiment effect: {significant_positive} ({significant_positive/total_companies*100:.1f}%)")
    print(f"Companies with significant NEGATIVE sentiment effect: {significant_negative} ({significant_negative/total_companies*100:.1f}%)")
    
    # Top positive correlations
    print(f"\n TOP 5 COMPANIES - Strongest Positive Sentiment Effects:")
    top_positive = results_df[results_df['pos_correlation'] > 0].head(5)
    for _, row in top_positive.iterrows():
        sig = "***" if row['pos_p_value'] < 0.001 else "**" if row['pos_p_value'] < 0.01 else "*" if row['pos_significant'] else ""
        print(f"  {row['ticker']} ({row['industry']}): r={row['pos_correlation']:.3f}{sig} (n={row['n_observations']})")
    
    # Most negative correlations
    print(f"\n BOTTOM 5 COMPANIES - Most Negative Sentiment Effects:")
    bottom_negative = results_df.nsmallest(5, 'pos_correlation')
    for _, row in bottom_negative.iterrows():
        sig = "***" if row['pos_p_value'] < 0.001 else "**" if row['pos_p_value'] < 0.01 else "*" if row['pos_significant'] else ""
        print(f"  {row['ticker']} ({row['industry']}): r={row['pos_correlation']:.3f}{sig} (n={row['n_observations']})")
    
    # Industry summary
    print(f"\n INDUSTRY PATTERNS:")
    industry_summary = results_df.groupby('industry').agg({
        'pos_correlation': ['mean', 'count'],
        'pos_significant': 'sum'
    }).round(3)
    industry_summary.columns = ['Avg_Correlation', 'N_Companies', 'N_Significant']
    industry_summary['Pct_Significant'] = (industry_summary['N_Significant'] / industry_summary['N_Companies'] * 100).round(1)
    
    for industry, stats in industry_summary.iterrows():
        print(f"  {industry}: Avg r={stats['Avg_Correlation']:.3f}, {stats['N_Significant']}/{stats['N_Companies']} significant ({stats['Pct_Significant']:.1f}%)")


def step9_company_specific_analysis():
    """Main function for company-specific analysis"""
    
    print("=" * 80)
    print("COMPANY-SPECIFIC SENTIMENT ANALYSIS")
    print("=" * 80)
    
    try:
        # Load data
        df = load_company_data()
        
        # Analyze each company
        results_df = analyze_company_correlations(df)
        
        # Create visualizations
        print("\nCreating company-specific visualizations...")
        company_plot_path = create_company_grid_plot(df, results_df)
        summary_plot_path = create_company_summary_plot(results_df)
        
        # Save results
        save_company_results(results_df)
        
        # Print findings
        print_key_findings(results_df)
        
        print("\n" + "=" * 80)
        print(" COMPANY-SPECIFIC ANALYSIS COMPLETED!")
        print("=" * 80)
        print("Files created:")
        print(f"• {company_plot_path}")
        print(f"• {summary_plot_path}")
        print("• Company_Specific_Analysis.csv")
        print("• Company_Specific_Analysis.xlsx")
        print("\nNow you can see which specific companies drive your results! 🎯")
        
    except Exception as e:
        print(f" Error: {str(e)}")
        import traceback
        traceback.print_exc()

step9_company_specific_analysis()

COMPANY-SPECIFIC SENTIMENT ANALYSIS
 Loaded data: 4121 observations for 58 companies
 Analyzed 58 companies with sufficient data

Creating company-specific visualizations...
 Creating individual company figures (3 plots each: Positive, Negative, Uncertainty)...
    Saved: TD_Sentiment_Analysis.png
    Saved: MDT_Sentiment_Analysis.png
    Saved: XOM_Sentiment_Analysis.png
    Saved: SAP_Sentiment_Analysis.png
    Saved: BAC_Sentiment_Analysis.png
    Saved: LMT_Sentiment_Analysis.png
    Saved: ABT_Sentiment_Analysis.png

  Created 7 individual company plots
 Location: exports/figures\company_specific
 Company summary plot saved: exports/figures\Company_Analysis_Summary.png
 Company results saved:
  - exports/tables\Company_Specific_Analysis.csv
  - exports/tables\Company_Specific_Analysis.xlsx

COMPANY-SPECIFIC SENTIMENT ANALYSIS RESULTS
Total companies analyzed: 58
Companies with significant POSITIVE sentiment effect: 3 (5.2%)
Companies with significant NEGATIVE sentiment effect: 0 (

# Step 9: Company-Specific Analysis

**Input Files:**
- `exports/tables/Final_Regression_Dataset.csv` (from Step 6)

**Output Files:**
- `exports/figures/company_specific/*.png` (individual company plots with 3 subplots each)
- `exports/figures/Company_Analysis_Summary.png`
- `exports/tables/Company_Specific_Analysis.csv`
- `exports/tables/Company_Specific_Analysis.xlsx`

**Analysis:**
- Individual company sentiment-CAR relationships
- 3 plots per company: Positive, Negative, Uncertainty vs CAR
- Correlation analysis by company and industry
This script creates a comprehensive overview visualization of key results.

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

def step10_create_summary_dashboard():
    """Create individual company dashboards for different industries"""
    
    # Set up the plotting style
    plt.style.use('default')
    sns.set_palette("husl")
    
    try:
        # Load final dataset
        df = pd.read_csv('exports/tables/Final_Regression_Dataset.csv')
        df['event_date'] = pd.to_datetime(df['event_date'])
        
        # Define industry classifications (same as used in the analysis)
        tech_companies = ['MSFT', 'GOOG', 'FB', 'INTC', 'ADBE', 'CRM', 'INTU', 'TSLA', 'ORCL', 'NFLX']
        financial_companies = ['BAC', 'JPM', 'MS', 'GS', 'AXP', 'BLK', 'V', 'MA', 'MCO']
        healthcare_companies = ['JNJ', 'ABT', 'LLY', 'AMGN', 'BMY', 'MRK', 'TMO', 'ISRG', 'MDT', 'UNH']
        energy_companies = ['XOM']
        
        # Create industry mapping
        industries = {
            'Technology': tech_companies,
            'Financial': financial_companies,
            'Healthcare': healthcare_companies,
            'Energy': energy_companies
        }
        
        # Select representative companies from each industry (companies with most data)
        selected_companies = {}
        
        for industry_name, company_list in industries.items():
            # Filter companies that exist in our dataset
            available_companies = [c for c in company_list if c in df['ticker'].unique()]
            
            if available_companies:
                # Select top 2-3 companies with most observations
                company_counts = df[df['ticker'].isin(available_companies)]['ticker'].value_counts()
                selected_companies[industry_name] = company_counts.head(3).index.tolist()
        
        print(f"[INFO] Selected representative companies:")
        for industry, companies in selected_companies.items():
            print(f"  {industry}: {companies}")
        
        # Create individual dashboards
        dashboard_count = 0
        
        for industry_name, company_list in selected_companies.items():
            for company in company_list:
                dashboard_count += 1
                create_company_dashboard(df, company, industry_name, dashboard_count)
        
        # Create industry comparison dashboard
        create_industry_comparison_dashboard(df, selected_companies)
        
        print(f"[OK] Created {dashboard_count} individual company dashboards plus 1 industry comparison")
        print("[OK] All dashboards saved in exports/figures/ directory")
        
    except Exception as e:
        print(f"[ERROR] Error creating dashboards: {e}")
        import traceback
        traceback.print_exc()

def create_company_dashboard(df, ticker, industry, dashboard_num):
    """Create individual dashboard for a specific company"""
    
    # Filter data for this company
    company_data = df[df['ticker'] == ticker].copy()
    
    if len(company_data) == 0:
        print(f"[WARNING] No data found for {ticker}")
        return
    
    # Create figure
    fig = plt.figure(figsize=(16, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.4, wspace=0.3)
    
    # Main title
    fig.suptitle(f'{ticker} ({industry} Industry) - Sentiment Analysis Dashboard', 
                 fontsize=20, fontweight='bold', y=0.95)
    
    # 1. Time series of CAR and sentiment
    ax1 = fig.add_subplot(gs[0, :])
    company_data_sorted = company_data.sort_values('event_date')
    
    ax1_twin = ax1.twinx()
    
    # Plot all available CAR windows
    lines = []
    car_colors = ['blue', 'navy', 'darkblue']
    car_markers = ['o', 's', '^']
    car_columns = []
    
    # Check which CAR windows are available
    if 'car_-1_1' in company_data_sorted.columns:
        car_columns.append(('car_-1_1', 'CAR(-1,+1)'))
    if 'car_0_1' in company_data_sorted.columns:
        car_columns.append(('car_0_1', 'CAR(0,+1)'))
    if 'car_0_2' in company_data_sorted.columns:
        car_columns.append(('car_0_2', 'CAR(0,+2)'))
    
    # If no specific CAR columns, use the default 'car' column
    if not car_columns and 'car' in company_data_sorted.columns:
        car_columns.append(('car', 'CAR(-1,+1)'))
    
    # Plot each CAR window
    for idx, (car_col, car_label) in enumerate(car_columns):
        if car_col in company_data_sorted.columns:
            line = ax1.plot(company_data_sorted['event_date'], company_data_sorted[car_col], 
                          color=car_colors[idx % len(car_colors)], marker=car_markers[idx % len(car_markers)],
                          alpha=0.7, linewidth=2, markersize=4, label=car_label)
            lines.extend(line)
    
    ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax1.set_ylabel('Cumulative Abnormal Return', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    
    # Plot positive sentiment on secondary axis
    line2 = ax1_twin.plot(company_data_sorted['event_date'], company_data_sorted['positive_ratio'], 
                          'g-s', alpha=0.7, linewidth=2, markersize=4, label='Positive Sentiment')
    ax1_twin.set_ylabel('Positive Sentiment Ratio', color='green')
    ax1_twin.tick_params(axis='y', labelcolor='green')
    
    ax1.set_title(f'{ticker} - Multiple CAR Windows vs Positive Sentiment Over Time', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Event Date')
    
    # Combine legends
    lines.extend(line2)
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper left', fontsize=9)
    
    # 2. Sentiment distribution for this company
    ax2 = fig.add_subplot(gs[1, 0])
    sentiment_data = [
        company_data['negative_ratio'].dropna(),
        company_data['positive_ratio'].dropna(),
        company_data['uncertainty_ratio'].dropna()
    ]
    
    if all(len(data) > 0 for data in sentiment_data):
        bp = ax2.boxplot(sentiment_data, tick_labels=['Negative', 'Positive', 'Uncertainty'], 
                        patch_artist=True)
        colors = ['lightcoral', 'lightgreen', 'lightskyblue']
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
    
    ax2.set_title(f'{ticker} Sentiment Distribution', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Sentiment Ratio')
    
    # 3. CAR distribution
    ax3 = fig.add_subplot(gs[1, 1])
    ax3.hist(company_data['car'].dropna(), bins=15, alpha=0.7, color='orange', edgecolor='black')
    ax3.axvline(company_data['car'].mean(), color='red', linestyle='--', 
               label=f'Mean: {company_data["car"].mean():.4f}')
    ax3.set_title(f'{ticker} CAR Distribution', fontsize=12, fontweight='bold')
    ax3.set_xlabel('CAR(-1,+1)')
    ax3.set_ylabel('Frequency')
    ax3.legend()
    
    # 4. Sentiment vs CAR scatter
    ax4 = fig.add_subplot(gs[1, 2])
    ax4.scatter(company_data['positive_ratio'], company_data['car'], 
               alpha=0.7, c='blue', s=50, edgecolors='black', linewidth=0.5)
    ax4.set_xlabel('Positive Sentiment Ratio')
    ax4.set_ylabel('CAR(-1,+1)')
    ax4.set_title(f'{ticker} Sentiment vs CAR', fontsize=12, fontweight='bold')
    
    # Add correlation
    if len(company_data) > 1:
        corr = company_data[['positive_ratio', 'car']].corr().iloc[0,1]
        ax4.text(0.05, 0.95, f'Correlation: {corr:.3f}', transform=ax4.transAxes, 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # 5. Performance metrics comparison
    ax5 = fig.add_subplot(gs[2, 0])
    
    # Compare this company vs industry average
    industry_companies = []
    if industry == 'Technology':
        industry_companies = ['MSFT', 'GOOG', 'FB', 'INTC', 'ADBE', 'CRM', 'INTU', 'TSLA', 'ORCL', 'NFLX']
    elif industry == 'Financial':
        industry_companies = ['BAC', 'JPM', 'MS', 'GS', 'AXP', 'BLK', 'V', 'MA', 'MCO']
    elif industry == 'Healthcare':
        industry_companies = ['JNJ', 'ABT', 'LLY', 'AMGN', 'BMY', 'MRK', 'TMO', 'ISRG', 'MDT', 'UNH']
    elif industry == 'Energy':
        industry_companies = ['XOM']
    
    industry_data = df[df['ticker'].isin(industry_companies)]
    
    metrics = ['CAR', 'Positive Sent.', 'Negative Sent.', 'Uncertainty']
    company_values = [
        company_data['car'].mean(),
        company_data['positive_ratio'].mean(),
        company_data['negative_ratio'].mean(),
        company_data['uncertainty_ratio'].mean()
    ]
    industry_values = [
        industry_data['car'].mean(),
        industry_data['positive_ratio'].mean(),
        industry_data['negative_ratio'].mean(),
        industry_data['uncertainty_ratio'].mean()
    ]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    ax5.bar(x - width/2, company_values, width, label=ticker, alpha=0.8, color='skyblue')
    ax5.bar(x + width/2, industry_values, width, label=f'{industry} Avg', alpha=0.8, color='lightcoral')
    
    ax5.set_xlabel('Metrics')
    ax5.set_ylabel('Average Values')
    ax5.set_title(f'{ticker} vs {industry} Industry', fontsize=12, fontweight='bold')
    ax5.set_xticks(x)
    ax5.set_xticklabels(metrics)
    ax5.legend()
    ax5.grid(axis='y', alpha=0.3)
    
    # 6. Statistics table
    ax6 = fig.add_subplot(gs[2, 1:])
    ax6.axis('off')
    
    # Create company statistics
    stats_data = [
        ['Observations', f"{len(company_data)}"],
        ['Time Period', f"{company_data['event_date'].min().strftime('%Y-%m')} to {company_data['event_date'].max().strftime('%Y-%m')}"],
        ['Average CAR', f"{company_data['car'].mean():.4f} ({company_data['car'].mean()*100:.2f}%)"],
        ['CAR Std Dev', f"{company_data['car'].std():.4f}"],
        ['Best Quarter CAR', f"{company_data['car'].max():.4f} ({company_data['car'].max()*100:.2f}%)"],
        ['Worst Quarter CAR', f"{company_data['car'].min():.4f} ({company_data['car'].min()*100:.2f}%)"],
        ['Avg Positive Sentiment', f"{company_data['positive_ratio'].mean():.4f}"],
        ['Avg Negative Sentiment', f"{company_data['negative_ratio'].mean():.4f}"],
        ['Avg Uncertainty', f"{company_data['uncertainty_ratio'].mean():.4f}"],
        ['Market Cap (Latest)', f"${company_data['market_cap'].iloc[-1]/1e9:.1f}B" if 'market_cap' in company_data.columns else 'N/A'],
    ]
    
    # Create table
    table = ax6.table(cellText=stats_data, 
                     colLabels=['Metric', 'Value'],
                     cellLoc='left',
                     loc='center',
                     colWidths=[0.4, 0.3])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)
    
    # Style the table
    for (i, j), cell in table.get_celld().items():
        if i == 0:  # Header row
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#4472C4')
            cell.set_text_props(color='white')
        else:
            cell.set_facecolor('#F2F2F2' if i % 2 == 0 else 'white')
    
    ax6.set_title(f'{ticker} Company Statistics', fontsize=14, fontweight='bold', pad=20)
    
    # Save the dashboard
    filename = f'Company_Dashboard_{dashboard_num:02d}_{ticker}_{industry}.png'
    plt.savefig(f'exports/figures/{filename}', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"[OK] Created dashboard for {ticker}: {filename}")

def create_industry_comparison_dashboard(df, selected_companies):
    """Create a dashboard comparing industries"""
    
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    fig.suptitle('Industry Comparison Dashboard - Sentiment Analysis', 
                 fontsize=24, fontweight='bold', y=0.95)
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Blue, Orange, Green, Red
    
    # 1. Average CAR by Industry
    ax1 = fig.add_subplot(gs[0, 0])
    industry_car = {}
    for industry, companies in selected_companies.items():
        industry_data = df[df['ticker'].isin(companies)]
        industry_car[industry] = industry_data['car'].mean()
    
    bars = ax1.bar(industry_car.keys(), industry_car.values(), color=colors[:len(industry_car)], alpha=0.8)
    ax1.set_title('Average CAR by Industry', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Average CAR(-1,+1)')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, industry_car.values()):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 2. Average Positive Sentiment by Industry
    ax2 = fig.add_subplot(gs[0, 1])
    industry_pos_sent = {}
    for industry, companies in selected_companies.items():
        industry_data = df[df['ticker'].isin(companies)]
        industry_pos_sent[industry] = industry_data['positive_ratio'].mean()
    
    bars = ax2.bar(industry_pos_sent.keys(), industry_pos_sent.values(), color=colors[:len(industry_pos_sent)], alpha=0.8)
    ax2.set_title('Average Positive Sentiment by Industry', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Positive Sentiment Ratio')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, industry_pos_sent.values()):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.0005, 
                f'{value:.4f}', ha='center', va='bottom', fontweight='bold')
    
    # 3. CAR Volatility by Industry
    ax3 = fig.add_subplot(gs[0, 2])
    industry_vol = {}
    for industry, companies in selected_companies.items():
        industry_data = df[df['ticker'].isin(companies)]
        industry_vol[industry] = industry_data['car'].std()
    
    bars = ax3.bar(industry_vol.keys(), industry_vol.values(), color=colors[:len(industry_vol)], alpha=0.8)
    ax3.set_title('CAR Volatility by Industry', fontsize=14, fontweight='bold')
    ax3.set_ylabel('CAR Standard Deviation')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, industry_vol.values()):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 4. Number of Observations by Industry
    ax4 = fig.add_subplot(gs[0, 3])
    industry_obs = {}
    for industry, companies in selected_companies.items():
        industry_data = df[df['ticker'].isin(companies)]
        industry_obs[industry] = len(industry_data)
    
    bars = ax4.bar(industry_obs.keys(), industry_obs.values(), color=colors[:len(industry_obs)], alpha=0.8)
    ax4.set_title('Observations by Industry', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Number of Earnings Calls')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, industry_obs.values()):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, 
                f'{value}', ha='center', va='bottom', fontweight='bold')
    
    # 5. Industry Sentiment Distribution
    ax5 = fig.add_subplot(gs[1, :2])
    
    industry_sentiment_data = []
    industry_labels = []
    
    for industry, companies in selected_companies.items():
        industry_data = df[df['ticker'].isin(companies)]
        industry_sentiment_data.append(industry_data['positive_ratio'].dropna())
        industry_labels.append(industry)
    
    bp = ax5.boxplot(industry_sentiment_data, tick_labels=industry_labels, patch_artist=True)
    for patch, color in zip(bp['boxes'], colors[:len(industry_labels)]):
        patch.set_facecolor(color)
        patch.set_alpha(0.8)
    
    ax5.set_title('Positive Sentiment Distribution by Industry', fontsize=14, fontweight='bold')
    ax5.set_ylabel('Positive Sentiment Ratio')
    ax5.grid(axis='y', alpha=0.3)
    
    # 6. Industry CAR Distribution
    ax6 = fig.add_subplot(gs[1, 2:])
    
    industry_car_data = []
    industry_labels = []
    
    for industry, companies in selected_companies.items():
        industry_data = df[df['ticker'].isin(companies)]
        industry_car_data.append(industry_data['car'].dropna())
        industry_labels.append(industry)
    
    bp = ax6.boxplot(industry_car_data, tick_labels=industry_labels, patch_artist=True)
    for patch, color in zip(bp['boxes'], colors[:len(industry_labels)]):
        patch.set_facecolor(color)
        patch.set_alpha(0.8)
    
    ax6.set_title('CAR Distribution by Industry', fontsize=14, fontweight='bold')
    ax6.set_ylabel('CAR(-1,+1)')
    ax6.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax6.grid(axis='y', alpha=0.3)
    
    # 7. Summary Statistics Table
    ax7 = fig.add_subplot(gs[2, :])
    ax7.axis('off')
    
    # Create comprehensive industry comparison table
    table_data = []
    headers = ['Industry', 'Companies', 'Observations', 'Avg CAR', 'CAR Std Dev', 
               'Avg Pos Sentiment', 'Avg Neg Sentiment', 'Avg Uncertainty']
    
    for industry, companies in selected_companies.items():
        industry_data = df[df['ticker'].isin(companies)]
        row = [
            industry,
            ', '.join(companies[:3]) + ('...' if len(companies) > 3 else ''),
            f"{len(industry_data)}",
            f"{industry_data['car'].mean():.4f}",
            f"{industry_data['car'].std():.4f}",
            f"{industry_data['positive_ratio'].mean():.4f}",
            f"{industry_data['negative_ratio'].mean():.4f}",
            f"{industry_data['uncertainty_ratio'].mean():.4f}"
        ]
        table_data.append(row)
    
    table = ax7.table(cellText=table_data, 
                     colLabels=headers,
                     cellLoc='center',
                     loc='center',
                     colWidths=[0.12, 0.18, 0.08, 0.08, 0.08, 0.12, 0.12, 0.12])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    # Style the table
    for (i, j), cell in table.get_celld().items():
        if i == 0:  # Header row
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#4472C4')
            cell.set_text_props(color='white')
        else:
            cell.set_facecolor(colors[i-1] if i-1 < len(colors) else '#F2F2F2')
            cell.set_alpha(0.3)
    
    ax7.set_title('Industry Comparison Summary', fontsize=16, fontweight='bold', pad=20)
    
    # Add methodology note
    methodology_text = """
Industry Analysis: Representative companies selected based on highest observation counts per industry.
Technology companies show different sentiment patterns compared to Financial and Healthcare sectors.
Each industry dashboard provides detailed analysis of top companies within that sector.
    """
    
    plt.figtext(0.5, 0.02, methodology_text, ha='center', fontsize=10, 
               style='italic', wrap=True)
    
    # Save the industry comparison dashboard
    plt.savefig('exports/figures/Industry_Comparison_Dashboard.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("[OK] Created Industry Comparison Dashboard: Industry_Comparison_Dashboard.png")


step10_create_summary_dashboard()
    

[INFO] Selected representative companies:
  Technology: ['MSFT', 'INTC', 'NFLX']
  Financial: ['MA', 'V', 'JPM']
  Healthcare: ['BMY', 'JNJ', 'AMGN']
  Energy: ['XOM']
[OK] Created dashboard for MSFT: Company_Dashboard_01_MSFT_Technology.png
[OK] Created dashboard for INTC: Company_Dashboard_02_INTC_Technology.png
[OK] Created dashboard for NFLX: Company_Dashboard_03_NFLX_Technology.png
[OK] Created dashboard for MA: Company_Dashboard_04_MA_Financial.png
[OK] Created dashboard for V: Company_Dashboard_05_V_Financial.png
[OK] Created dashboard for JPM: Company_Dashboard_06_JPM_Financial.png
[OK] Created dashboard for BMY: Company_Dashboard_07_BMY_Healthcare.png
[OK] Created dashboard for JNJ: Company_Dashboard_08_JNJ_Healthcare.png
[OK] Created dashboard for AMGN: Company_Dashboard_09_AMGN_Healthcare.png
[OK] Created dashboard for XOM: Company_Dashboard_10_XOM_Energy.png
[OK] Created Industry Comparison Dashboard: Industry_Comparison_Dashboard.png
[OK] Created 10 individual company dash

# Step 10: Summary Dashboard

**Input Files:**
- `exports/tables/Final_Regression_Dataset.csv` (from Step 6)

**Output Files:**
- `exports/figures/Summary_Dashboard.png` (comprehensive overview)

**Dashboard Components:**
- Overall sentiment-CAR relationships
- Industry comparisons
- Temporal patterns
- Key statistics and findings

# Step 11: Report Figures for Final Publication

**Input Files:**
- `exports/tables/Final_Regression_Dataset.csv` (from Step 6)

**Output Files:**
- `exports/figures/report_figures/Table1_Descriptive_Statistics.csv`
- `exports/figures/report_figures/Table1_Descriptive_Statistics.png`
- `exports/figures/report_figures/Table2_Correlation_Matrix.csv`
- `exports/figures/report_figures/Figure1_Correlation_Matrix.png`
- `exports/figures/report_figures/Figure2_Sentiment_CAR_Scatterplots.png`
- `exports/figures/report_figures/Table3_Industry_Statistics.csv`
- `exports/figures/report_figures/Figure3_Industry_CAR_Boxplot.png`
- `exports/figures/report_figures/Figure4_Control_Variables_Scatterplots.png`
- `exports/figures/report_figures/Table4_Regression_Results.csv`
- `exports/figures/report_figures/Table5_Model_Statistics.csv`
- `exports/figures/report_figures/Table6_Period_Statistics.csv`
- `exports/figures/report_figures/Figure5_Period_Comparison.png`

**Publication-Ready Figures:**
1. Descriptive statistics table
2. Correlation matrix heatmap
3. Sentiment vs CAR scatterplots (positive, negative, uncertainty)
4. Industry comparison boxplots
5. Control variables analysis
6. OLS regression results
7. COVID vs Non-COVID period comparison

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import os

# Set style for professional figures
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9

# Load data
data_path = "exports/tables/Final_Regression_Dataset.csv"
df = pd.read_csv(data_path)

print("=" * 80)
print("COMPREHENSIVE VISUALIZATION FOR FINAL REPORT")
print("=" * 80)
print(f"\nDataset loaded: {len(df)} observations from {df['ticker'].nunique()} companies")
print(f"Date range: {df['event_date'].min()} to {df['event_date'].max()}")

# Create output directory for figures
figure_dir = "exports/figures/report_figures"
os.makedirs(figure_dir, exist_ok=True)
print(f"Figures will be saved to: {figure_dir}")

# ========================================
# 1️ DESCRIPTIVE STATISTICS TABLE
# ========================================

print("\n" + "=" * 80)
print("1 DESCRIPTIVE STATISTICS")
print("=" * 80)

descriptive_cols = [
    "car", "positive_ratio", "negative_ratio", "uncertainty_ratio",
    "price_volatility_pre_event", "log_market_cap",
    "price_momentum_1month", "abnormal_turnover", "total_words"
]

# Create descriptive statistics table
desc_stats = df[descriptive_cols].describe().T
desc_stats['var'] = df[descriptive_cols].var()
desc_stats = desc_stats[['mean', 'std', 'var', 'min', '25%', '50%', '75%', 'max']]

# Rename for better presentation
desc_stats.index = [
    'CAR (-1,+1)',
    'Positive Sentiment Ratio',
    'Negative Sentiment Ratio',
    'Uncertainty Ratio',
    'Price Volatility (Pre-Event)',
    'Log Market Cap',
    'Price Momentum (1 Month)',
    'Abnormal Turnover (%)',
    'Total Words'
]

print("\nDescriptive Statistics (All Companies):")
print(desc_stats.round(4))

# Save to CSV
desc_path = os.path.join(figure_dir, "Table1_Descriptive_Statistics.csv")
desc_stats.to_csv(desc_path)
print(f"\n Saved: {desc_path}")

# Create a visual table
fig, ax = plt.subplots(figsize=(14, 6))
ax.axis('tight')
ax.axis('off')

table_data = desc_stats.round(4).reset_index()
table_data.columns = ['Variable', 'Mean', 'Std Dev', 'Variance', 'Min', '25%', 'Median', '75%', 'Max']

table = ax.table(cellText=table_data.values,
                colLabels=table_data.columns,
                cellLoc='center',
                loc='center',
                colWidths=[0.25] + [0.09]*8)

table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)

# Color header
for i in range(len(table_data.columns)):
    table[(0, i)].set_facecolor('#4472C4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Alternate row colors
for i in range(1, len(table_data) + 1):
    for j in range(len(table_data.columns)):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#E7E6E6')
        else:
            table[(i, j)].set_facecolor('#F2F2F2')

plt.title('Table 1: Descriptive Statistics (All Companies)', 
          fontsize=14, fontweight='bold', pad=20)

table_fig_path = os.path.join(figure_dir, "Table1_Descriptive_Statistics.png")
plt.savefig(table_fig_path, bbox_inches='tight', dpi=300)
plt.close()
print(f" Saved: {table_fig_path}")

# ========================================
# 2 CORRELATION MATRIX
# ========================================

print("\n" + "=" * 80)
print("2 CORRELATION MATRIX")
print("=" * 80)

correlation_cols = [
    "car", "positive_ratio", "negative_ratio", "uncertainty_ratio",
    "price_volatility_pre_event", "log_market_cap",
    "price_momentum_1month", "abnormal_turnover"
]

# Compute correlation matrix
corr_matrix = df[correlation_cols].corr()

# Create figure
fig, ax = plt.subplots(figsize=(12, 10))

# Create heatmap
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
cmap = sns.diverging_palette(250, 10, as_cmap=True)

sns.heatmap(corr_matrix, 
            mask=mask,
            annot=True, 
            fmt='.3f',
            cmap=cmap,
            center=0,
            square=True,
            linewidths=1,
            cbar_kws={"shrink": 0.8, "label": "Correlation Coefficient"},
            ax=ax,
            vmin=-1, vmax=1)

# Better labels
labels = [
    'CAR\n(-1,+1)',
    'Positive\nRatio',
    'Negative\nRatio',
    'Uncertainty\nRatio',
    'Price\nVolatility',
    'Log Market\nCap',
    'Price\nMomentum',
    'Abnormal\nTurnover'
]

ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_yticklabels(labels, rotation=0)

plt.title('Figure 1: Correlation Matrix of Key Variables (All Companies)', 
          fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
corr_path = os.path.join(figure_dir, "Figure1_Correlation_Matrix.png")
plt.savefig(corr_path, bbox_inches='tight', dpi=300)
plt.close()

print(f"\n Saved: {corr_path}")

# Save correlation matrix to CSV
corr_csv_path = os.path.join(figure_dir, "Table2_Correlation_Matrix.csv")
corr_matrix.to_csv(corr_csv_path)
print(f" Saved: {corr_csv_path}")

# ========================================
# 3 SCATTER PLOTS: SENTIMENT vs CAR
# ========================================

print("\n" + "=" * 80)
print("3 SENTIMENT VS CAR SCATTER PLOTS")
print("=" * 80)

# Create 3 scatterplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sentiment_vars = [
    ('positive_ratio', 'Positive Sentiment Ratio', '#2E7D32'),
    ('negative_ratio', 'Negative Sentiment Ratio', '#C62828'),
    ('uncertainty_ratio', 'Uncertainty Ratio', '#F57C00')
]

for idx, (var, title, color) in enumerate(sentiment_vars):
    ax = axes[idx]
    
    # Scatter plot
    ax.scatter(df[var], df['car'], alpha=0.5, s=30, color=color, edgecolors='white', linewidth=0.5, zorder=2)
    
    # Add regression line - FIXED: more visible with proper styling
    valid_mask = df[var].notna() & df['car'].notna()
    if valid_mask.sum() > 1:
        z = np.polyfit(df.loc[valid_mask, var], df.loc[valid_mask, 'car'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(df[var].min(), df[var].max(), 100)
        ax.plot(x_line, p(x_line), 
                color='black', 
                linewidth=3, 
                linestyle='--', 
                dashes=(5, 3),  # Explicit dash pattern
                label='Regression Line',
                zorder=3,  # Draw on top
                alpha=0.8)
    
    # Calculate correlation
    corr = df[[var, 'car']].corr().iloc[0, 1]
    
    # Add correlation annotation
    ax.text(0.05, 0.95, f'ρ = {corr:.3f}', 
            transform=ax.transAxes, fontsize=11, fontweight='bold',
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    ax.set_xlabel(title, fontsize=11, fontweight='bold')
    ax.set_ylabel('CAR (-1, +1)', fontsize=11, fontweight='bold')
    ax.set_title(f'{title} vs CAR', fontsize=12, fontweight='bold', pad=10)
    ax.grid(True, alpha=0.3, zorder=1)
    ax.legend(loc='lower right', framealpha=0.9)

plt.suptitle('Figure 2: Sentiment Ratios vs Cumulative Abnormal Returns (All Companies)', 
             fontsize=14, fontweight='bold', y=1.02)

plt.tight_layout()
scatter_path = os.path.join(figure_dir, "Figure2_Sentiment_CAR_Scatterplots.png")
plt.savefig(scatter_path, bbox_inches='tight', dpi=300)
plt.close()

print(f"\n Saved: {scatter_path}")

# Print correlations
print("\nSentiment-CAR Correlations:")
for var, title, _ in sentiment_vars:
    corr = df[[var, 'car']].corr().iloc[0, 1]
    print(f"  {title}: {corr:.4f}")

# ========================================
# 4 INDUSTRY COMPARISON (Boxplot)
# ========================================

print("\n" + "=" * 80)
print("4 INDUSTRY COMPARISON")
print("=" * 80)

# Create industry label from dummy variables
industry_cols = ["tech", "financial", "healthcare", "energy", "consumer", "other"]
df_industry = df.copy()

# Map industry dummies to labels
def get_industry(row):
    if row['tech'] == 1:
        return 'Technology'
    elif row['financial'] == 1:
        return 'Financial'
    elif row['healthcare'] == 1:
        return 'Healthcare'
    elif row['energy'] == 1:
        return 'Energy'
    elif row['consumer'] == 1:
        return 'Consumer'
    else:
        return 'Other'

df_industry['industry'] = df_industry.apply(get_industry, axis=1)

# Create boxplot
fig, ax = plt.subplots(figsize=(12, 7))

# Define colors for each industry
industry_colors = {
    'Technology': '#2196F3',
    'Financial': '#4CAF50',
    'Healthcare': '#F44336',
    'Energy': '#FF9800',
    'Consumer': '#9C27B0',
    'Other': '#607D8B'
}

# Create boxplot with custom colors
industries = ['Technology', 'Financial', 'Healthcare', 'Energy', 'Consumer', 'Other']
positions = range(len(industries))
box_data = [df_industry[df_industry['industry'] == ind]['car'].dropna() for ind in industries]

bp = ax.boxplot(box_data, positions=positions, labels=industries,
                patch_artist=True, widths=0.6,
                boxprops=dict(linewidth=1.5),
                medianprops=dict(linewidth=2, color='darkred'),
                whiskerprops=dict(linewidth=1.5),
                capprops=dict(linewidth=1.5))

# Color boxes
for patch, industry in zip(bp['boxes'], industries):
    patch.set_facecolor(industry_colors[industry])
    patch.set_alpha(0.7)

# Add mean markers
means = [data.mean() for data in box_data]
ax.plot(positions, means, 'D', color='black', markersize=8, 
        label='Mean', markerfacecolor='yellow', markeredgewidth=2)

# Add horizontal line at y=0
ax.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)

ax.set_xlabel('Industry Sector', fontsize=12, fontweight='bold')
ax.set_ylabel('CAR (-1, +1)', fontsize=12, fontweight='bold')
ax.set_title('Figure 3: Distribution of CAR by Industry Sector (All Companies)', 
             fontsize=14, fontweight='bold', pad=15)
ax.grid(True, alpha=0.3, axis='y')
ax.legend(loc='upper right')

plt.xticks(rotation=15, ha='right')
plt.tight_layout()

boxplot_path = os.path.join(figure_dir, "Figure3_Industry_CAR_Boxplot.png")
plt.savefig(boxplot_path, bbox_inches='tight', dpi=300)
plt.close()

print(f"\n Saved: {boxplot_path}")

# Print industry statistics
print("\nCAR Statistics by Industry:")
industry_stats = df_industry.groupby('industry')['car'].agg(['count', 'mean', 'std', 'min', 'max'])
industry_stats = industry_stats.round(4)
print(industry_stats)

# Save to CSV
industry_stats_path = os.path.join(figure_dir, "Table3_Industry_Statistics.csv")
industry_stats.to_csv(industry_stats_path)
print(f"\n Saved: {industry_stats_path}")

# ========================================
# 5 CONTROL EFFECTS (Scatter Plots)
# ========================================

print("\n" + "=" * 80)
print("5 CONTROL VARIABLE EFFECTS")
print("=" * 80)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

control_vars = [
    ('log_market_cap', 'Log Market Capitalization', '#1976D2'),
    ('price_volatility_pre_event', 'Price Volatility (Pre-Event)', '#D32F2F')
]

for idx, (var, title, color) in enumerate(control_vars):
    ax = axes[idx]
    
    # Scatter plot
    ax.scatter(df[var], df['car'], alpha=0.5, s=30, color=color, edgecolors='white', linewidth=0.5)
    
    # Add regression line
    valid_data = df[[var, 'car']].dropna()
    if len(valid_data) > 0:
        z = np.polyfit(valid_data[var], valid_data['car'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(df[var].min(), df[var].max(), 100)
        ax.plot(x_line, p(x_line), color='darkred', linewidth=2, linestyle='--', label='Regression Line')
        
        # Calculate correlation
        corr = valid_data.corr().iloc[0, 1]
        
        # Add correlation annotation
        ax.text(0.05, 0.95, f'ρ = {corr:.3f}', 
                transform=ax.transAxes, fontsize=11, fontweight='bold',
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    ax.set_xlabel(title, fontsize=11, fontweight='bold')
    ax.set_ylabel('CAR (-1, +1)', fontsize=11, fontweight='bold')
    ax.set_title(f'{title} vs CAR', fontsize=12, fontweight='bold', pad=10)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best')

plt.suptitle('Figure 4: Control Variables vs Cumulative Abnormal Returns (All Companies)', 
             fontsize=14, fontweight='bold', y=1.02)

plt.tight_layout()
control_path = os.path.join(figure_dir, "Figure4_Control_Variables_Scatterplots.png")
plt.savefig(control_path, bbox_inches='tight', dpi=300)
plt.close()

print(f"\n Saved: {control_path}")

# ========================================
# 6 REGRESSION ANALYSIS (OLS)
# ========================================

print("\n" + "=" * 80)
print("6 REGRESSION ANALYSIS")
print("=" * 80)

# Prepare regression data
dependent_var = "car"
independent_vars = [
    "positive_ratio", "negative_ratio", "uncertainty_ratio",
    "price_volatility_pre_event", "log_market_cap",
    "price_momentum_1month", "abnormal_turnover",
    "tech", "financial", "healthcare", "energy", "consumer"
]

# Create regression dataset (drop rows with missing values)
reg_data = df[[dependent_var] + independent_vars].dropna()

print(f"\nRegression sample size: {len(reg_data)} observations")

# Prepare X and y
X = reg_data[independent_vars]
y = reg_data[dependent_var]

# Add constant
X = sm.add_constant(X)

# Run OLS regression with robust standard errors
model = sm.OLS(y, X)
results = model.fit(cov_type='HC3')

print("\n" + "=" * 80)
print("REGRESSION RESULTS (Heteroskedasticity-Robust Standard Errors)")
print("=" * 80)
print(results.summary())

# Create regression results table
reg_table = pd.DataFrame({
    'Variable': results.params.index,
    'Coefficient': results.params.values,
    'Std Error': results.bse.values,
    't-statistic': results.tvalues.values,
    'P-value': results.pvalues.values,
    'Significant': ['***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.1 else '' 
                    for p in results.pvalues.values]
})

reg_table = reg_table.round(4)
print("\n" + "=" * 80)
print("Regression Coefficients Table:")
print("=" * 80)
print(reg_table)

# Save regression results
reg_results_path = os.path.join(figure_dir, "Table4_Regression_Results.csv")
reg_table.to_csv(reg_results_path, index=False)
print(f"\n Saved: {reg_results_path}")

# Model statistics
model_stats = pd.DataFrame({
    'Statistic': ['R-squared', 'Adj. R-squared', 'F-statistic', 'Prob (F-statistic)', 'No. Observations'],
    'Value': [results.rsquared, results.rsquared_adj, results.fvalue, results.f_pvalue, results.nobs]
})
print("\n" + "=" * 80)
print("Model Statistics:")
print("=" * 80)
print(model_stats)

model_stats_path = os.path.join(figure_dir, "Table5_Model_Statistics.csv")
model_stats.to_csv(model_stats_path, index=False)
print(f" Saved: {model_stats_path}")

# ========================================
# 7 PERIOD COMPARISON (COVID vs Non-COVID)
# ========================================

print("\n" + "=" * 80)
print("7 PERIOD COMPARISON (COVID vs Non-COVID)")
print("=" * 80)

if 'covid_period' in df.columns:
    # Calculate mean CAR by period
    period_stats = df.groupby('covid_period')['car'].agg(['count', 'mean', 'std']).reset_index()
    period_stats.columns = ['Period', 'Count', 'Mean CAR', 'Std Dev']
    period_stats['Period'] = period_stats['Period'].map({0: 'Non-COVID', 1: 'COVID'})
    
    print("\nCAR by Period:")
    print(period_stats)
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = ['#4CAF50', '#F44336']
    bars = ax.bar(period_stats['Period'], period_stats['Mean CAR'], 
                  color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
    
    # Add error bars (standard deviation)
    ax.errorbar(period_stats['Period'], period_stats['Mean CAR'], 
                yerr=period_stats['Std Dev'], fmt='none', 
                ecolor='black', capsize=10, capthick=2)
    
    # Add value labels on bars
    for i, (bar, val) in enumerate(zip(bars, period_stats['Mean CAR'])):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                f'{val:.4f}',
                ha='center', va='bottom', fontweight='bold', fontsize=11)
    
    # Add sample size labels
    for i, (bar, count) in enumerate(zip(bars, period_stats['Count'])):
        ax.text(bar.get_x() + bar.get_width()/2., -0.005,
                f'n = {count}',
                ha='center', va='top', fontsize=10, style='italic')
    
    ax.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax.set_ylabel('Mean CAR (-1, +1)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Period', fontsize=12, fontweight='bold')
    ax.set_title('Figure 5: CAR Comparison - COVID vs Non-COVID Period (All Companies)', 
                 fontsize=14, fontweight='bold', pad=15)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    period_path = os.path.join(figure_dir, "Figure5_Period_Comparison.png")
    plt.savefig(period_path, bbox_inches='tight', dpi=300)
    plt.close()
    
    print(f"\n Saved: {period_path}")
    
    # T-test for difference
    covid_car = df[df['covid_period'] == 1]['car'].dropna()
    non_covid_car = df[df['covid_period'] == 0]['car'].dropna()
    
    t_stat, p_value = stats.ttest_ind(covid_car, non_covid_car)
    print(f"\nT-test for difference in means:")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_value:.4f}")
    print(f"  Significant: {'Yes' if p_value < 0.05 else 'No'}")
    
    # Save period statistics
    period_stats_path = os.path.join(figure_dir, "Table6_Period_Statistics.csv")
    period_stats.to_csv(period_stats_path, index=False)
    print(f" Saved: {period_stats_path}")
else:
    print(" 'covid_period' column not found in dataset. Skipping period comparison.")

# ========================================
# FINAL SUMMARY
# ========================================

print("\n" + "=" * 80)
print(" ALL VISUALIZATIONS COMPLETED!")
print("=" * 80)
print(f"\nAll figures and tables saved to: {figure_dir}")
print("\nGenerated files:")
print("  Tables:")
print("    - Table1_Descriptive_Statistics.csv & .png")
print("    - Table2_Correlation_Matrix.csv")
print("    - Table3_Industry_Statistics.csv")
print("    - Table4_Regression_Results.csv")
print("    - Table5_Model_Statistics.csv")
if 'covid_period' in df.columns:
    print("    - Table6_Period_Statistics.csv")
print("\n  Figures:")
print("    - Figure1_Correlation_Matrix.png")
print("    - Figure2_Sentiment_CAR_Scatterplots.png")
print("    - Figure3_Industry_CAR_Boxplot.png")
print("    - Figure4_Control_Variables_Scatterplots.png")
if 'covid_period' in df.columns:
    print("    - Figure5_Period_Comparison.png")
print("\n" + "=" * 80)


COMPREHENSIVE VISUALIZATION FOR FINAL REPORT

Dataset loaded: 4285 observations from 59 companies
Date range: 2005-10-31 00:40:28 to 2021-03-12 01:38:07
Figures will be saved to: exports/figures/report_figures

1 DESCRIPTIVE STATISTICS

Descriptive Statistics (All Companies):
                                   mean        std           var       min  \
CAR (-1,+1)                      0.0031     0.0508  2.600000e-03   -0.3485   
Positive Sentiment Ratio         0.0352     0.0092  1.000000e-04    0.0072   
Negative Sentiment Ratio         0.0217     0.0076  1.000000e-04    0.0022   
Uncertainty Ratio                0.0126     0.0045  0.000000e+00    0.0000   
Price Volatility (Pre-Event)     0.0172     0.0115  1.000000e-04    0.0029   
Log Market Cap                  24.7043     0.9576  9.170000e-01   20.4489   
Price Momentum (1 Month)         0.0202     0.0871  7.600000e-03   -0.5263   
Abnormal Turnover (%)           87.9224   124.2784  1.544512e+04  -68.4688   
Total Words          